# HuGS-DW: Human GWAS-SNP Cross-Reference Data Warehouse
#### Este notebook e todos os ficheiros acessórios, podem ser directamente acedidos em:
https://drive.google.com/open?id=1rTqXz1AE8mW-uZWK81W0BnNA2YmE11JZ
### 1. Dimensions and facts tables of the data warehouse

### a. Define and model them in SQL
##### O "star schema" actualizado do nosso DW, é o seguinte:


![alt text](HuGs_Star_Schema.png)

As tabelas de dimensões e factos identificadas na fase 1 do projecto, foram actualizadas (com adição de novas tabelas de dimensões auxiliares, como a Allele_Dimension e mudanças de medidas associadas às tabelas de factos) e modeladas em SQL.

### Tabelas de Dimensões

* **Gender_Dimension**- Contém dados sobre esperança média de vida e rendimento nacional bruto (em dólares) para indivíduos de diferentes sexos, distribuídos pelos diferentes países nos quais se distribuem os indíviduos do estudo. Agrega uma primary key (gender_key) com outras variáveis como o country code, country e gender. Foram ainda adicionados indíces demográficos como Average Income e  Mortality rate, ambos de 2015 (data da publicação do estudo do 1000 genomes project). Esta dimensão vai permitir obter informação demográfica, diferenciada por género e país.

* **Population_Dimension** - Nesta dimensão agrega-se informação acerca de cada uma das populações incluídas no estudo genético do 1000 genomes. Esta dimensão contém informação para: Population key (PK), Population Code, Population Description, Super Population Code, Super Population (East Asian, South Asian, African, European, American), Ethinicity, Country code, Country, Region (e.g. Utah) e City. Inclui também dados demográficos, como Life Expectancy e Gross Income (ambos relativos ao ano de 2015, ano de publicação do estudo). Esta informação demográfica foi também modelada em classes e adicionada à tabela de dimensão.

* **Person_Dimension**- Dimensão modelada com: Person key(PK), Person  code, Population Code, Population Description, Super Population Code, Super Population, Country Code, Country e Gender. Esta dimensão agrega a informação sobre cada uma das pessoas estudadas no 1000 genomes project.

* **Chromosome_Dimension**- Aqui, agrupa-se a informação sobre os cromossomas do genoma humano. Esta dimensão tem informação para Choromosome Key(PK), Chromosome Number, RefSeqID, Size(Mb), GC%, protein, rRNA, tRNA, other RNA, gene, pseudogene,  Number of genes per mb. 

* **Allele_Dimension**- Esta "role-playing" dimension, foi criada para ajudar na inteligibilidade do esquema de dados. Contém apenas duas linhas, quer permitem identificar os alelos como de referência ou alternativos. Visto que os dados de variantes genéticas do 1000 genomes estão codificados como 0, para um alelo de referência e 1, para um alelo alternativo, esta dimensão vai permitir identificar cada tipo de alelo dos indivíduos. Contém unicamente uma Allele Key e Allele Type.

* **SNP_Dimension**- Esta dimensão contém informações acerca dos SNPs presentes no cromossoma 19, nomeadamente, a referência, dados acerca da sua localização e contexto genómico, informações acerca da frequência em diferentes populações e consequências moleculares associadas. A tabela de dimensão tem os seguintes atributos: SNP Key(PK), chromosome, snp position,  snp ref code, ReferenceAllele, AlternativeAllele, East Asian frequency, European frequency, African frequency, American frequency, South ASian frequency.

* **Disease_Dimension**- Dimensão onde se agrega a informação acerca de doenças humanas (da “Human Disease Ontology” - DOID). Contêm  Disease key(PK), wikidata_id, Disease_name, Doid_ID, Mesh_ID, Type_of_Disease, Health_specialities, Symptoms, Genetic_Association, Organs affected, Caused, Alternative_names. 

* **Study_Dimension**- Esta dimensão inclui informação provenient de estudos de GWAS e agregados no site GWAS Catalog (https://www.ebi.ac.uk/gwas/). Agrupa informação sobre Study key (PK), Study,  Date, Disease, Region, Chromosome Position, Reported genes, Mapped_genes, Upstream Gene ID, Downstream Gene ID, SNP Gene IDS, Upstream Gene Distance, Dowstream Gene Distance, Strongest SNP Risk Allele , SNPs , Context, Intergenic, Risk Allele Frequency, p -value. Em suma, nesta dimensão existem vários parâmetros estatísticos para a associação de uma doença a uma determinada variante genética.

### Tabelas de Factos
* **SNP Person Fact**- Esta tabela de factos mudou relativamente ao nosso esquema inicial. É agora uma factless table, que contém informação acerca do genótipo em cada um dos SNPs de cada um dos 2503 indivíduos estudados. Inclui chaves estrangeiras que vão permitir ligar os factos, às diferentes dimensões: Gender Key, Population Key, Person Key, Chromossome Key, SNP Key, Allele 1 Key, Allele 2 Key (estas duas últimas colunas, ligam à mesma tabela de Allele_Dimension).

* **Disease GWAS fact**- Esta tabela vai conter informação atómica acerca da associação genética entre uma doença e um SNP,  identificada num determinado estudo. Mudamos esta fact table, para incluir uma medida não aditiva, o p-value, que efectivamente mede a a validade probabilística da associação entre uma doença e um SNP. Inclui chaves estrangeiras que ligam às várias dimensões, como SNP Key, Disease Keye Study Key e ainda uma medida não aditiva, p-value.

### b. Identify hierarchies and fact granularity
##### Granularidade das tabelas de facto
- A granularidade dos factos não mudaram relativamente à primeira fase.
- **SNP Person Fact**- Nesta tabela cada linha corresponde à variante de um dado SNP para um dado indivíduo. Inclui o genótipo para os dois alelos do indivíduo e referências para tabelas de dimensões contendo informações sobre esse indivíduo, população, género, cromossoma e informações sobre o SNP.
- **Disease GWAS Fact**- Nesta tabela cada linha corresponde a uma referência num estudo que associa um SNP a uma doença e a respectiva medida probabilística dessa associação.

##### Hierarquias do sistema de dados
- Dentro das várias tabelas de dimensões, não há hierarquias óbvias que necessitem de uma modelação mais elaborada. 
- A única hierarquia que existe nos nossos dados, prende-se com informação geográfica na Population Dimension. Com o objectivo de manter a legibilidade da estrutura de dados, para o end-user, as hierarquias são mantidas na mesma tabela. 
- Com o crescimento do número de indíviduos estudados, poder-se-iam contemplar criar outtriggers para dados demográficos e organizar melhor a hierarquia geográfica. Presentemente, tendo em conta a dimensão do dataset, esta modelação não se justifica.

### c. Create the dimensions and facts tables in the DBMS (postgreSQL)
- Em baixo colocamos o código usado para modelar, filtrar, inserir dados e executar instruções SQL, que são utilizados ao longo de todo o notebook. 

In [2]:
import psycopg2 as pg
import pandas as pd
import os

def pandafy(rows, colnames):
    N=len(colnames)
    D={cn: [] for cn in colnames}
    for row in rows:
        for i in range(N): D[colnames[i]].append(row[i])
    pdfy=pd.DataFrame(D)
    return pdfy

def excuteSingleSQLstatement(sql, host, database, user, password):
    conn = pg.connect(host=host,database=database, user=user, password=password)
    cur = conn.cursor()
    cur.execute(sql)
    cur.close()
    conn.commit()
    conn.close()    

def getSQLfromQuery(sql, params, host, database, user, password):
    conn = pg.connect(host=host, database=database, user=user, password=password)
    cur  = conn.cursor()    
    if len(params)==0:
        cur.execute(sql)
    else: 
        cur.execute(sql, params)
    data = cur.fetchall()
    colnames = [desc[0] for desc in cur.description]
    cur.close()
    conn.close()
    df=pandafy(data, colnames)
    return df

- Em baixo, estão os comandos SQL para criar as várias tabelas de dimensões e factos.

In [134]:
# ATTENTION!!!!
# Run only if you want to DROP all tables from the database!!!

SQL = '''
drop table if exists chromosome_dimension;
drop table if exists person_dimension;
drop table if exists population_dimension;
drop table if exists gender_dimension;
drop table if exists allele_dimension;
drop table if exists disease_dimension;
drop table if exists snp_dim;
drop table if exists study_dim;
drop table if exists disease_gwas_fact;
drop table if exists snp_person_fact

'''

# Remove # from the line below,  if you want to drop all tables.
excuteSingleSQLstatement(SQL, host="appserver-01.alunos.di.fc.ul.pt",database="tpd020", user="tpd020", password="tpd020_1000genomes")

In [135]:
SQL = '''
create table chromosome_dimension(
chromosome_key int,
chromosome varchar(2),
refseq varchar(12), 
size_Mb float,
GC_percentage float,
proteins numeric,
rRNA  numeric,
tRNA numeric,
other_RNA numeric,
gene numeric,
pseudogene numeric,
genes_per_MB float,
primary key (chromosome_key)
);

create table person_dimension(
person_key int,
person_code varchar(8),
population_code char(3),
population_description varchar(128), 
super_population_code char(3),
super_population varchar(128),
country_code char(3),
country varchar(128),
gender varchar(8),
primary key (person_key)
);

create table population_dimension(
population_key int,
population_code char(3),
population_description varchar(128), 
super_population_code char(3),
super_population varchar(128),
ethnicity varchar(32),
country_code char(3),
country varchar(128),
region varchar(128),
city varchar(128),
gross_national_income int,
gross_national_income_group varchar(24),
life_expectancy_birth float,
life_expectancy_birth_group varchar(24),
primary key (population_key)
);

create table gender_dimension(
gender_key int,
country_code char(3),
country varchar(128),
gender varchar(8),
literacy_rate float,
unemployment_rate float,
mortality_per_1000_habitants float,
primary key (gender_key)
);

create table allele_dimension(
allele_key int,
allele_type_code char(3),
allele_type varchar(12),
primary key (allele_key)
);

create table disease_dimension(
disease_key int,
wikidata_id varchar(30),
disease_name varchar(160),
doid_id int, 
mesh_id varchar(20),
type_of_disease varchar(1760),
health_speciality  varchar(160),
symptoms varchar(1860),
genetic_associations varchar(60) ARRAY,
affects varchar(280),
caused_by varchar(480),
alternative_names varchar(20000),
primary key (disease_key)
);

create table snp_dim(
snp_id serial primary key,
snp_ref_code varchar(64),
gene varchar(16),
functional_consequence varchar(512), --arrays?
clinical_significance varchar(512), -- arrays?
chromosome float,
snp_position varchar(32),
reference char(128),
alternative char(128),
info_string varchar(1024),
context varchar(64),
easian_frequency float,
european_frequency float,
african_frequency float,
american_frequency float,
sasian_frequency float,
variation_type varchar(34),
consequence varchar(35),
mapped_gene varchar(36),
upstream_gene_id varchar(37), 
downstream_gene_id varchar(38),
upstream_gene_distance varchar(39),
dowstream_gene_distance varchar(40),
region varchar(41),
chr_pos varchar(42)
);

create table study_dim(
study_id serial primary key,
pumbmed_id int,
study_date date,
disease varchar(256),
region varchar(32),
chr_pos varchar(64),
reported_genes varchar(120) array,
mapped_genes varchar(121) array,
upstream_gene_id varchar(122),
downstream_gene_id varchar(123),
snp_gene_ids varchar(512) array,
upstream_gene_distance varchar(124),
dowstream_gene_distance varchar(125),
strongest_snp_risk_allele varchar(126),
snps varchar(127),
snp_id_current varchar(128),
context varchar(129),
intergenic varchar(130),
risk_allele_frequency varchar(131),
p_value float
);

--facts
create table snp_person_fact(
gender_id integer REFERENCES gender_dimension(gender_key),
population_id integer REFERENCES population_dimension(population_key),
snp_id integer REFERENCES snp_dim(snp_id),
person_id integer REFERENCES person_dimension(person_key),
chromossome_id integer REFERENCES chromosome_dimension(chromosome_key),
allele_1 int REFERENCES allele_dimension(allele_key), 
allele_2 int REFERENCES allele_dimension(allele_key)
);

create table disease_gwas_fact(
disease_id integer REFERENCES disease_dimension(disease_key),
snp_id integer REFERENCES snp_dim(snp_id),
study_id integer REFERENCES study_dim(study_id),
p_value float
)
'''

excuteSingleSQLstatement(SQL, host="appserver-01.alunos.di.fc.ul.pt",database="tpd020", user="tpd020", password="tpd020_1000genomes")

### 2. Define an ETL workflow

### a. Identify all data sources for all dimensions. Add URL links to all data that should be available. If not public data, point to dropbox files, Google drive, or whatever

O preenchimento das várias dimensões deste DW, implicou um uso de um grande número de datasets distintos, que foram processados, filtrados e agregados em várias tabelas durante o processo de ETL.
Assim, por dimensão, os datasets usados foram:

#### Person_Dimension
- Esta dimensão, contendo informação acerca de cada um dos indíviduos cujo genoma foi sequenceado, está contida nos FTPs do 1000 genomes project.
- Dataset com informação acerca dos indivíduos analisados no estudo: ftp://ftp.1000genomes.ebi.ac.uk/vol1/ftp/release/20130502/integrated_call_samples_v3.20130502.ALL.panel
- Informação acerca das populações dos indivíduos estudados: ftp://ftp.1000genomes.ebi.ac.uk/vol1/ftp/phase3/20131219.populations.tsv
- Informação acerca das superpopulações dos indivíduos estudados: ftp://ftp.1000genomes.ebi.ac.uk/vol1/ftp/phase3/20131219.superpopulations.tsv

#### Population_Dimension
- Esta dimensão contém informação acerca das populações analisadas no estudo dos 1000 genomes project. A tabela foi manualmente curada, visto ter uma dimensão reduzida.
- Informação acerca das populações dos indivíduos estudados: ftp://ftp.1000genomes.ebi.ac.uk/vol1/ftp/phase3/20131219.populations.tsv
- Tabela modificada das populações: https://drive.google.com/open?id=1TQdAgXMhoDs3CBjnOIFuEcJDjat3U704bJtktkNSnOo
- Para informação demográfica acerca dos países, usamos os dados do "The World Bank" (https://data.worldbank.org/)
- Gross Income: https://data.worldbank.org/indicator/NY.GNP.PCAP.PP.CD
- Gross Income (Google Drive direct download): https://drive.google.com/open?id=1u0pDvbPSGk0xGdrJksvlC6ia3F54yxTyviA98CT1chw
- Life expectancy at birth: https://data.worldbank.org/indicator/sp.dyn.le00.in
- Life expectancy at birth (Google Drive direct download): https://drive.google.com/open?id=1BNiG-yshJJgKSv9P5qXbp6nWHKnmS7S6

#### Gender_Dimension
- Esta dimensão agrega informação acerca do género e de indíces demográficos em cada um dos países em que o estudo foi realizado.
- A informação acerca do género, é derivada das tabelas de indivíduos e populações descritas em cima. A estas adiciona-se dados do "The World Bank".
- Mortality rate per 1,000 adults: for female- https://data.worldbank.org/indicator/SP.DYN.AMRT.FE for male https://data.worldbank.org/indicator/SP.DYN.AMRT.MA
- Unemployment per 1,000 adults: for female -https://data.worldbank.org/indicator/SL.UEM.TOTL.FE.ZS for male https://data.worldbank.org/indicator/SL.UEM.TOTL.MA.ZS?view=chart
- Literacy rate : for female-https://data.worldbank.org/indicator/SE.ADT.LITR.FE.ZS for male -https://data.worldbank.org/indicator/SE.ADT.LITR.MA.ZS 

#### Chromosome_Dimension
- Dimensão que agrega informação acerca dos vários cromossomas nos quais se distribuem os SNPs identificados no estudo.
- Apesar deste exercício académico só conter a informação genética do cromossoma 19 (devido à dimensão dos datasets), a informação de todos os cromossomas foi agregada nesta dimensão.
- Tabela com informação acerca dos cromossomas do genoma humano: https://www.ncbi.nlm.nih.gov/genome/?term=human[organism]
- Link para csv criada: https://drive.google.com/open?id=1hUzEmuz-3nwMRg-VHmmR8NJUvthkjvl7

#### Allele_Dimension
- Esta pequena dimensão role-playing, agrega informação acerca do tipo de variação encontrada no genótipo de cada indíviduo. Em suma, dá informação acerca da presença de uma alelo de referência ou alternativo em cada cromossoma.
- A informação genética para cada indíviduo analizado no estudo, para o cromossoma 19, encontra-se em: ftp://ftp.1000genomes.ebi.ac.uk/vol1/ftp/release/20130502/ALL.chr19.phase3_shapeit2_mvncall_integrated_v5a.20130502.genotypes.vcf.gz

#### SNP_Dimension
- Esta dimensão agrega vários tipos de informação acerca dos SNPs no cromossoma 19. Combina vários datasets:
- Informação genética dos 1000 genomes project: ftp://ftp.1000genomes.ebi.ac.uk/vol1/ftp/release/20130502/ALL.chr19.phase3_shapeit2_mvncall_integrated_v5a.20130502.genotypes.vcf.gz
- Informação retirada do dbSNP: https://www.ncbi.nlm.nih.gov/snp/?term=9%5BChromosome%5D
- Informação retirada do GWAS dataset: https://www.ebi.ac.uk/gwas/api/search/downloads/alternative

#### Study_Dimension
- Esta dimensão contém informação de estudos de Genome-Wide Association Studies.
- Contém informação retirada do GWAS association dataset: https://www.ebi.ac.uk/gwas/api/search/downloads/alternative

#### Disease_Dimension
- Aqui guardam-se as doenças humanas para as quais existe um identificador Doid (Human Disease Ontology).
- Esta informação foi particularmente difícil de agregar (por falta de uma ferramenta que agregue esta informação). No final, a informação foi descarregada do Wikidata, usando SPARQL.
- Informação acerca de todas as doenças com um doidID: https://tinyurl.com/y9hu7nvb
- Informação acerca de termos alternativos para as doenças: https://tinyurl.com/yd2zwg69

### b. For each dimension show the code used for modeling, filtering and inserting data

# Chromosome_Dimension

### Código para ler e filtrar informação acerca de cromossomas humanos

In [15]:
import csv

def load_csv(csv_file):
    '''This function reads a csv file with information about human chromosomes.
    It compiles the information about each chromosome into a list of lists.
    It corrects value types. It also calculates the gene density for each chromosome.
    Requires: A csv file with information about each chromosome.
    Ensures: A list of lists, with all modeled information about chromosomes.
    '''
    
    with open(csv_file, newline='') as csv_file:
        csv_reader = csv.reader(csv_file, delimiter=',')
        data = list(csv_reader)

        chromosomes = []
        i = 1
        
        for chr in data[1:25]:
            key = i
            chrom = chr[2]
            refSeq = chr[3]
            size = float(chr[5])
            GC = float(chr[6])
            prot = int(chr[7].replace(',', ''))
            if chr[8].isdigit():
                rRNA = int(chr[8])
            else:
                rRNA = 0
            if chr[9].isdigit():
                tRNA = int(chr[9])
            else:
                tRNA = 0
            if chr[10].isdigit():
                otherRNA = int(chr[10].replace(',', ''))
            else:
                otherRNA = 0                
            gene = int(chr[11].replace(',', ''))
            if chr[12].isdigit():
                pseudogene = int(chr[12].replace(',', ''))
            else:
                pseudogene = 0                
            genesPerMb = round(gene/size, 2)
            
            i += 1
            
            chromosomes.append([key, chrom, refSeq, size, GC, prot, rRNA, tRNA, otherRNA, gene, pseudogene, genesPerMb])
     
    return chromosomes

chromosomes = (load_csv("Chromosome_Dimension_DB.csv"))

### Inserção e verificação dos dados na Chromosome_Dimension

In [ ]:
#SQL = """drop table chromosome_dimension"""
SQL= """create table chromosome_dimension(
	chromosome_key int,
	chromosome varchar(2),
	refseq varchar(12), 
	size_Mb float,
	GC_percentage float,
    proteins numeric,
    rRNA  numeric,
    tRNA numeric,
    other_RNA numeric,
    gene numeric,
    pseudogene numeric,
    genes_per_MB float,
	primary key (chromosome_key)
)"""

excuteSingleSQLstatement(SQL, host="appserver-01.alunos.di.fc.ul.pt",database="tpd020", user="tpd020", password="tpd020_1000genomes")

In [17]:
# Este código assume que estamos a inserir os dados no servidor pgadmin do DI da FCUL.
conn = pg.connect(host="appserver-01.alunos.di.fc.ul.pt",database="tpd020", user="tpd020", password="tpd020_1000genomes")
curs = conn.cursor()
for chr in chromosomes:
    key, chrom, refSeq, size, GC, prot, rRNA, tRNA, otherRNA, gene, pseudogene, genesPerMb = chr
    curs.execute("insert into chromosome_dimension values(%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)", (key, chrom, refSeq, size, GC, prot, rRNA, tRNA, otherRNA, gene, pseudogene, genesPerMb))
curs.close()
conn.commit()
conn.close()

In [18]:
SQL = "select * from chromosome_dimension"
df  = getSQLfromQuery(SQL, (), host="appserver-01.alunos.di.fc.ul.pt",database="tpd020", user="tpd020", password="tpd020_1000genomes")
df

,chromosome_key,chromosome,refseq,size_mb,gc_percentage,proteins,rrna,trna,other_rna,gene,pseudogene,genes_per_mb
0,1,1,NC_000001.11,248.96,42.3,11096,17,90,0,5101,0,20.49
1,2,2,NC_000002.12,242.19,40.3,8455,0,7,0,3879,0,16.02
2,3,3,NC_000003.12,198.30,39.7,7242,0,4,0,2988,909,15.07
3,4,4,NC_000004.12,190.22,38.3,4646,0,1,0,2439,805,12.82
4,5,5,NC_000005.10,181.54,39.5,4803,0,17,0,2595,789,14.29
5,6,6,NC_000006.12,170.81,39.6,5630,0,138,0,3019,892,17.67
6,7,7,NC_000007.14,159.35,40.7,5220,0,22,0,2774,914,17.41
7,8,8,NC_000008.11,145.14,40.2,4087,0,4,0,2174,680,14.98
8,9,9,NC_000009.12,138.40,42.3,4679,0,3,0,2269,723,16.39
9,10,10,NC_000010.11,133.80,41.6,5449,0,3,0,2178,643,16.28


# Person_Dimension

### Código para ler, filtrar e modelar dados acerca de 2503 indivíduos do estudo 1000 genomes project.

In [19]:
import csv

def load_csv(pop_csv_file, csv_file):
    '''
    This function loads the file with information about the populations and a file with information about individuals.
    It compiles this information into a new list, ready to insert into the Person_Dimension table in pgAdmin.
    Requires: A population table and an individuals table, following a specific format.
    Ensures: A list of individuals studied, with information about population, superpopulation and gender.
    '''

    with open(pop_csv_file, newline='') as csv_file1:
        csv_reader = csv.reader(csv_file1, delimiter="\t")
        popData = list(csv_reader)

        population = []
        i = 1
        
        for pop in popData[1:27]:
            key = i
            popCode = pop[0]
            popDescript = pop[1]
            superPopCode = pop[2]
            superPop = pop[3]
            ethnicity = pop[4]
            countryCode = pop[5]
            country = pop[6]
            
            i += 1
            
            population.append([key, popCode, popDescript, superPopCode, superPop, ethnicity, countryCode, country])
            
    with open(csv_file, newline='') as csv_file2:
        csv_reader = csv.reader(csv_file2, delimiter="\t")
        data = list(csv_reader)

        person = []
        i = 1
        
        for ppl in data[1:]:
            key = i
            individual = ppl[0]
            popCode = ppl[1]
            for pop in population:
                if popCode == pop[1]:
                    popDescript = pop[2]
            superPopCode = ppl[2]
            for pop in population:
                if superPopCode == pop[3]:
                    superPop = pop[4]
            for pop in population:
                if popCode == pop[1]:
                    countryCode = pop[6]
            for pop in population:
                if countryCode == pop[6]:
                    country = pop[7]
            gender = ppl[3]
            
            i += 1
            
            person.append([key, individual, popCode, popDescript, superPopCode, superPop, countryCode, country, gender])

    return person

people = load_csv('20131219.populations - 20131219.populations.tsv', 'integrated_call_samples_v3.20130502.ALL.panel')

### Inserção e verificação dos dados na Person_Dimension

In [10]:
#SQL = """drop table person_dimension"""
SQL= """create table person_dimension(
	person_key int,
	person_code varchar(8),
	population_code char(3),
    population_description varchar(128), 
	super_population_code char(3),
	super_population varchar(128),
    country_code char(3),
    country varchar(128),
    gender varchar(8),
    primary key (person_key)
)"""

excuteSingleSQLstatement(SQL, host="appserver-01.alunos.di.fc.ul.pt",database="tpd020", user="tpd020", password="tpd020_1000genomes")

In [20]:
conn = pg.connect(host="appserver-01.alunos.di.fc.ul.pt",database="tpd020", user="tpd020", password="tpd020_1000genomes")
curs = conn.cursor()
for ppl in people:
    key, individual, popCode, popDescript, superPopCode, superPop, countryCode, country, gender = ppl
    curs.execute("insert into person_dimension values(%s, %s, %s, %s, %s, %s, %s, %s, %s)", (key, individual, popCode, popDescript, superPopCode, superPop, countryCode, country, gender))
curs.close()
conn.commit()
conn.close()

In [21]:
SQL = "select * from person_dimension"
df  = getSQLfromQuery(SQL, (), host="appserver-01.alunos.di.fc.ul.pt",database="tpd020", user="tpd020", password="tpd020_1000genomes")
df

,person_key,person_code,population_code,population_description,super_population_code,super_population,country_code,country,gender
0,1,HG00096,GBR,British in England and Scotland,EUR,European,GBR,United Kingdom,male
1,2,HG00097,GBR,British in England and Scotland,EUR,European,GBR,United Kingdom,female
2,3,HG00099,GBR,British in England and Scotland,EUR,European,GBR,United Kingdom,female
3,4,HG00100,GBR,British in England and Scotland,EUR,European,GBR,United Kingdom,female
4,5,HG00101,GBR,British in England and Scotland,EUR,European,GBR,United Kingdom,male
...,...,...,...,...,...,...,...,...,...
2499,2500,NA21137,GIH,"Gujarati Indian in Houston,TX",SAS,South Asian,USA,United States of America,female
2500,2501,NA21141,GIH,"Gujarati Indian in Houston,TX",SAS,South Asian,USA,United States of America,female
2501,2502,NA21142,GIH,"Gujarati Indian in Houston,TX",SAS,South Asian,USA,United States of America,female
2502,2503,NA21143,GIH,"Gujarati Indian in Houston,TX",SAS,South Asian,USA,United States of America,female


# Population_Dimension

### Código para ler, filtrar e modelar dados acerca das populações.

#### Exemplo de um dos ficheiros do "The World Bank" e sua visualização.

In [22]:
#step 1. read base data
lines=open("API_NY.GNP.PCAP.CD_DS2_en_csv_v2_936138.csv", "rt").readlines()

#step 2. remove the first 4 lines as it only contain metadata and write all over again
#        so that we may benefit from Pandas read_csv function
fil=open("country_data.csv", "wt")
for lin in lines[4:]: fil.write(lin)
fil.close()

#step 3. read the data for pandas queries
df=pd.read_csv("country_data.csv")
df.head()

,Country Name,Country Code,Indicator Name,Indicator Code,1960,1961,1962,1963,1964,1965,...,2011,2012,2013,2014,2015,2016,2017,2018,2019,Unnamed: 64
0,Aruba,ABW,"GNI per capita, Atlas method (current US$)",NY.GNP.PCAP.CD,NaN,NaN,NaN,NaN,NaN,NaN,...,22450.0,23520.0,23970.0,24500.0,23780.0,23550.0,23630.0,NaN,NaN,NaN
1,Afghanistan,AFG,"GNI per capita, Atlas method (current US$)",NY.GNP.PCAP.CD,NaN,NaN,NaN,NaN,NaN,NaN,...,530.0,630.0,660.0,630.0,600.0,570.0,550.0,550.0,NaN,NaN
2,Angola,AGO,"GNI per capita, Atlas method (current US$)",NY.GNP.PCAP.CD,NaN,NaN,NaN,NaN,NaN,NaN,...,3420.0,4170.0,4780.0,5010.0,4520.0,3770.0,3560.0,3370.0,NaN,NaN
3,Albania,ALB,"GNI per capita, Atlas method (current US$)",NY.GNP.PCAP.CD,NaN,NaN,NaN,NaN,NaN,NaN,...,4410.0,4360.0,4540.0,4540.0,4390.0,4320.0,4290.0,4860.0,NaN,NaN
4,Andorra,AND,"GNI per capita, Atlas method (current US$)",NY.GNP.PCAP.CD,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


#### Load the "Growth National Income" (in dolars) and the "Life Expectancy at Birth" into memory.

In [23]:
#### Load the "Growth National Income" (in dolars) and the "Life Expectancy at Birth" into memory.

import csv

def load_csv(csv_file):
    '''This function reads a csv file from "The World Bank", and extracts the information for a certain year
    for all the countries.
    Requires: csv file with demographic indexes for all countries.
    Ensures: List with countries and respective index for the year of 2015, the year the 1000 genomes project
    was published.
    '''
    
    with open(csv_file, newline='') as csv_file:
        csv_reader = csv.reader(csv_file, delimiter=",")
        data = list(csv_reader)

        countryList = []
        i = 1
        
        for country in data[5:]:
            key = i
            cntry = country[0]
            countryCode = country[1]
            GNI2015 = country[59]
                        
            i +=1
            
            countryList.append([key, cntry, countryCode, GNI2015])
   
    return countryList

GNI = load_csv('API_NY.GNP.PCAP.CD_DS2_en_csv_v2_936138.csv')
LEB2015 = load_csv('API_SP.DYN.LE00.IN_DS2_en_csv_v2_935946.csv')

In [24]:
import csv

# Define age groups fo life expectancy
def ExpectancyClass(expec):
    if expec <=60  : return "Below 60yo"
    if expec <=75 : return "60yo - 75yo"
    return "Above 75"
    
# Define gross national income per capita per GNI brackets 
def IncomeClass(gni_pc):
    if gni_pc <=1000 : return "Below 1k"
    if gni_pc <=4000  : return "1k - 4k"
    if gni_pc <=14000  : return "4k - 14k"
    return "Above 14k"

def load_csv(csv_file):
    
    with open(csv_file, newline='') as csv_file:
        csv_reader = csv.reader(csv_file, delimiter="\t")
        data = list(csv_reader)

        population = []
        i = 1
        
        for pop in data[1:27]:
            key = i
            popCode = pop[0]
            popDescript = pop[1]
            superPopCode = pop[2]
            superPop = pop[3]
            ethnicity = pop[4]
            countryCode = pop[5]
            country = pop[6]
            region = pop[7]
            city = pop[8]
            for ctry in GNI:
                if countryCode == ctry[2]:
                    grossNationalIncome = float(ctry[3])
            for ctry in LEB2015:
                if countryCode == ctry[2]:
                    lifeExpectancyBirth = round(float(ctry[3]), 2)
            i += 1
            
            population.append([key, popCode, popDescript, superPopCode, superPop, ethnicity, countryCode, country, region, city, grossNationalIncome, IncomeClass(grossNationalIncome), lifeExpectancyBirth, ExpectancyClass(lifeExpectancyBirth)
            ])
         
    return population

population = (load_csv('20131219.populations - 20131219.populations.tsv'))

### Inserção e verificação dos dados na Population_Dimension

In [22]:
#SQL = """drop table population_dimension"""
SQL= """create table population_dimension(
	population_key int,
	population_code char(3),
	population_description varchar(128), 
	super_population_code char(3),
	super_population varchar(128),
    ethnicity varchar(32),
    country_code char(3),
    country varchar(128),
    region varchar(128),
    city varchar(128),
    gross_national_income int,
    gross_national_income_group varchar(24),
    life_expectancy_birth float,
    life_expectancy_birth_group varchar(24),
    primary key (population_key)
)"""

excuteSingleSQLstatement(SQL, host="appserver-01.alunos.di.fc.ul.pt",database="tpd020", user="tpd020", password="tpd020_1000genomes")

In [25]:
conn = pg.connect(host="appserver-01.alunos.di.fc.ul.pt",database="tpd020", user="tpd020", password="tpd020_1000genomes")
curs = conn.cursor()
for pop in population:
    key, popCode, popDescript, superPopCode, superPop, ethnicity, countryCode, country, region, city, grossNationalIncome, IncomeClass, lifeExpectancyBirth, ExpectancyClass = pop
    curs.execute("insert into population_dimension values(%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)", (key, popCode, popDescript, superPopCode, superPop, ethnicity, countryCode, country, region, city, grossNationalIncome, IncomeClass, lifeExpectancyBirth, ExpectancyClass))
curs.close()
conn.commit()
conn.close()

In [26]:
SQL = "select * from population_dimension"
df  = getSQLfromQuery(SQL, (), host="appserver-01.alunos.di.fc.ul.pt",database="tpd020", user="tpd020", password="tpd020_1000genomes")
df

,population_key,population_code,population_description,super_population_code,super_population,ethnicity,country_code,country,region,city,gross_national_income,gross_national_income_group,life_expectancy_birth,life_expectancy_birth_group
0,1,CDX,"Chinese Dai in Xishuangbanna, China",EAS,East Asian,Dai,CHN,China,Xishuangbanna,,7910,4k - 14k,75.93,Above 75
1,2,CHB,"Han Chinese in Beijing, China",EAS,East Asian,Han,CHN,China,Beijing,Beijing,7910,4k - 14k,75.93,Above 75
2,3,JPT,"Japanese in Tokyo, Japan",EAS,East Asian,Japanese,JPN,Japan,Kantō,Tokyo,38840,Above 14k,83.79,Above 75
3,4,KHV,"Kinh in Ho Chi Minh City, Vietnam",EAS,East Asian,Kinh,VNM,Vietnam,Ho Chi Minh,Ho Chi Minh,1970,1k - 4k,75.11,Above 75
4,5,CHS,"Southern Han Chinese, China",EAS,East Asian,Han,CHN,China,Southern China,,7910,4k - 14k,75.93,Above 75
5,6,BEB,Bengali in Bangladesh,SAS,South Asian,Bengali,BGD,Bangladesh,Bangladesh,,1220,1k - 4k,71.51,60yo - 75yo
6,7,GIH,"Gujarati Indian in Houston,TX",SAS,South Asian,Gujarati,USA,United States of America,"Houston, Texas",Houston,56720,Above 14k,78.69,Above 75
7,8,ITU,Indian Telugu in the UK,SAS,South Asian,Telugu,GBR,United Kingdom,United Kingdom,,44280,Above 14k,80.96,Above 75
8,9,PJL,"Punjabi in Lahore,Pakistan",SAS,South Asian,Punjabi,PAK,Pakistan,Punjab,Lahore,1360,1k - 4k,66.58,60yo - 75yo
9,10,STU,Sri Lankan Tamil in the UK,SAS,South Asian,Tamil,GBR,United Kingdom,United Kingdom,,44280,Above 14k,80.96,Above 75


# Gender_Dimension

### Código para ler, filtrar e modelar dados demográficos relativos ao género .

In [27]:
import csv

def load_csv(csv_file):
    '''This function reads a csv file from "The World Bank", and extracts the information for a certain year
    for all the countries.
    Requires: csv file with demographic indexes for all countries.
    Ensures: List with countries and respective index for the year of 2015, the year the 1000 genomes project
    was published.
    '''
    
    with open(csv_file , newline='') as csv_file:
        csv_reader = csv.reader(csv_file, delimiter=",")
        data = list(csv_reader)

        countryList = []
        i = 1        
        for country in data[5:]:
            key = i
            cntry = country[0]
            countryCode = country[1]
            LT2015 = country[59]
                        
            i +=1
            
            countryList.append([key, cntry, countryCode, LT2015])

    return countryList

In [28]:
# Literacy rate for male and females per country (% of male and females, ages 15-24, for the year of 2015).
LTM = load_csv("API_SE.ADT.LT.MA.ZS_DS2_en_csv_v2_1000301.csv")
LTF = load_csv("API_SE.ADT.1524.LT.FE.ZS_DS2_en_csv_v2_989948.csv")

# Unemployement rate for male and female per country s for the year of 2015.
UNEMM = load_csv("API_SL.UNEM.TOTL.MA.ZS_DS2_en_csv_v2_995787.csv")
UNEMF = load_csv("API_SL.UNEM.TOTL.FE.ZS_DS2_en_csv_v2_1002674.csv")

In [29]:
def load_csv(csv_file):
    '''This function reads a csv file from "The World Bank" with mortality indexes.It extracts the information for a certain year
    for all the countries.
    Requires: csv file with demographic indexes for all countries.
    Ensures: List with countries and respective index for the year of 2015, the year the 1000 genomes project
    was published.
    '''
    
    with open(csv_file , newline='') as csv_file:
        csv_reader = csv.reader(csv_file, delimiter=",")
        data = list(csv_reader)
        #print(data)

        countryList = []
        i = 1        
        for country in data[5:]:
            key = i
            cntry = country[2]
            countryCode = country[3]
            MOR = country[11]
                        
            i +=1
            
            countryList.append([key, cntry, countryCode, MOR])
        
    return countryList

In [30]:
#MORTALITY  rate female/male per 1000 habitants, for the year of 2015.
MORTM=load_csv("Mort_MA-1f90-45b0-992b-3231bcd2c05b_Data.csv")
MORTF=load_csv("Mort_FE-020e-4949-8810-d261d40a218a_Data.csv")

In [31]:
import csv

# This function identifies the countries found in our population dataset and creates a list with the countries found.
# This list will be used to create the information to upload to the Gender_Dimension.
def load_csv(csv_file):
    '''This function identifies the countries found in our population dataset and creates a list with the countries found.
    This list contains information, per country, of several indexes for male and females.
    This list will be used to create the information to upload to the Gender_Dimension.
    Requires: A csv file with information about population and lists (previously loaded), with information about several
    demographic indexes.
    Ensures: List of demographic indexes per sex, per country.
    '''
    
    with open(csv_file, newline='') as csv_file:
        csv_reader = csv.reader(csv_file, delimiter="\t")
        data = list(csv_reader)

        countries = []
        
        for pop in data[1:27]:
            countryCode = pop[5]
            country = pop[6]
            
            if [countryCode, country] not in countries:
                countries.append([countryCode, country])
        
        countries.sort()
    
    gender = []
    
    i = 1
    for country in countries:
        keyMale = i
        keyFemale = i+1
        countryCode = country[0]
        country = country[1]
        for lit in LTM:
            if countryCode == lit[2]:
                if lit[3] != '':
                    maleLiteracy = float(lit[3])
                else:
                    maleLiteracy = 0                    
        for lit in LTF:
            if countryCode == lit[2]:
                if lit[3] != '':
                    femaleLiteracy = float(lit[3])
                else:
                    femaleLiteracy = 0
        for unemp in UNEMM:
            if countryCode == unemp[2]:
                if unemp[3] != '':
                    maleUnemployement = float(unemp[3])
                else:
                    maleUnemployement = 0                
        for unemp in UNEMF:
            if countryCode == unemp[2]:
                if unemp[3] != '':
                    femaleUnemployement = float(unemp[3])
                else:
                    femaleUnemployement = 0 
        for mort in MORTM:
            if countryCode == mort[2]:
                if mort[3] != '' and mort[3] != '..':
                    maleMortality = float(mort[3])
                else:
                    maleMortality = 0 
        for mort in MORTF:
            if countryCode == mort[2]:
                if mort[3] != '' and mort[3] != '..':
                    femaleMortality = float(mort[3])
                else:
                    femaleMortality = 0
                    
        i += 2

        gender.append([keyMale, countryCode, country, 'male', round(maleLiteracy, 2), round(maleUnemployement, 2), round(maleMortality, 2)])
        gender.append([keyFemale, countryCode, country, 'female', round(femaleLiteracy, 2), round(femaleUnemployement, 2), round(femaleMortality, 2)])
        
    return gender

gender = load_csv('20131219.populations - 20131219.populations.tsv')

### Inserção e verificação dos dados na Gender_Dimension

In [47]:
#SQL = """drop table gender_dimension"""
SQL= """create table gender_dimension(
	gender_key int,
    country_code char(3),
    country varchar(128),
    gender varchar(6),
    literacy_rate float,
    unemployment_rate float,
    mortality_per_1000_habitants float,
    primary key (gender_key)
)"""

excuteSingleSQLstatement(SQL, host="appserver-01.alunos.di.fc.ul.pt",database="tpd020", user="tpd020", password="tpd020_1000genomes")

In [32]:
conn = pg.connect(host="appserver-01.alunos.di.fc.ul.pt",database="tpd020", user="tpd020", password="tpd020_1000genomes")
curs = conn.cursor()
for gen in gender:
    key, gender, countryCode, country, literacy, unemployment, mortality_rate_1000_habi = gen
    curs.execute("insert into gender_dimension values(%s, %s, %s, %s, %s, %s, %s)", (key, gender, countryCode, country, literacy, unemployment, mortality_rate_1000_habi))
curs.close()
conn.commit()
conn.close()

# As there are lots of values for literacy_rate missing from "The World Bank" data, we will transform these values, that were inserted as 0.00, into NULL values.
conn = pg.connect(host="appserver-01.alunos.di.fc.ul.pt",database="tpd020", user="tpd020", password="tpd020_1000genomes")
curs = conn.cursor()
curs.execute("UPDATE gender_dimension SET literacy_rate = NULL WHERE literacy_rate = 0.00")
curs.close()
conn.commit()
conn.close()

In [33]:
SQL = "select * from gender_dimension order by gender_key"
df  = getSQLfromQuery(SQL, (), host="appserver-01.alunos.di.fc.ul.pt",database="tpd020", user="tpd020", password="tpd020_1000genomes")
df

,gender_key,country_code,country,gender,literacy_rate,unemployment_rate,mortality_per_1000_habitants
0,1,BGD,Bangladesh,male,68.06,3.16,150.04
1,2,BGD,Bangladesh,female,89.54,7.47,109.47
2,3,BRB,Barbados,male,NaN,12.35,124.02
3,4,BRB,Barbados,female,NaN,10.32,74.17
4,5,CHN,China,male,NaN,5.11,100.07
5,6,CHN,China,female,NaN,4.01,64.48
6,7,COL,Colombia,male,94.06,6.36,190.84
7,8,COL,Colombia,female,98.92,10.84,89.30
8,9,ESP,Spain,male,98.77,20.78,77.00
9,10,ESP,Spain,female,99.69,23.55,38.81


# Allele_Dimension

- Esta pequena dimensão "role-playing", vai servir para interpretar o valor dos genótipos de cada indivíduo, em cada SNP, em cada linha de facto.
- Após inserção dos factos na tabela SNP Person Fact, os valores de genotipagem para os diferentes alelos de 0 ou 1, serão interpretados como "allele_is_reference" para valores de 0, ou "allele_is_alternative", para valores de 1.

In [ ]:
#SQL = """drop table allele_dimension"""
SQL= """create table allele_dimension(
	allele_key int,
    allele_type_code char(3),
    allele_type varchar(12),
    primary key (allele_key)
)"""

excuteSingleSQLstatement(SQL, host="appserver-01.alunos.di.fc.ul.pt",database="tpd020", user="tpd020", password="tpd020_1000genomes")

### Inserção e verificação dos dados na Allele_Dimension
- Esta dimensão só tem duas linhas, que identificam o alelo como de referência, ou alternativo.

In [35]:
conn = pg.connect(host="appserver-01.alunos.di.fc.ul.pt",database="tpd020", user="tpd020", password="tpd020_1000genomes")
curs = conn.cursor()
curs.execute("insert into allele_dimension values (0, 'Ref', 'Reference')")
curs.execute("insert into allele_dimension values (1, 'Alt', 'Alternative')")             
curs.close()
conn.commit()
conn.close()

In [36]:
SQL = "select * from allele_dimension"
df  = getSQLfromQuery(SQL, (), host="appserver-01.alunos.di.fc.ul.pt",database="tpd020", user="tpd020", password="tpd020_1000genomes")
df

,allele_key,allele_type_code,allele_type
0,0,Ref,Reference
1,1,Alt,Alternative


# Disease_Dimension

- Começamos por extrair a informação do Wikidata Query Service. Dada a dimensão dos dados e da query, tivemos que dividir a query em duas sub-queries e depois juntá-las.
- Usamos os seguintes comandos SPARQL:

In [ ]:
# ATENÇÃO: Este comando é para introduzir no Wikidata Query service, encontrado em: https://query.wikidata.org/
# Dado a dimensão da Query, tivemos que deixar alguns campos de fora, e dividir a query 
# Este comando dá-nos todas as doenças com um DOID identifier, mas não conseguimos obter nomes alternativos,  pois o query resultava num timeout.

PREFIX dbo: <http://dbpedia.org/ontology/>
# disesases
SELECT 
?item  
?itemName
?DoidID
?MeSHID
(GROUP_CONCAT(DISTINCT ?subClassLabel ; SEPARATOR = ",") AS  ?subClass) 
(GROUP_CONCAT(DISTINCT ?healthspecialtyLabel ; SEPARATOR = ",") AS  ?healthspecialties) 
(GROUP_CONCAT(DISTINCT ?symptomLabel ; SEPARATOR = ",") AS  ?symptoms) 
(GROUP_CONCAT(DISTINCT ?geneticassociationLabel ; SEPARATOR = ",") AS  ?geneticassociations) 
(GROUP_CONCAT(DISTINCT ?afflictLabel ; SEPARATOR = ",") AS  ?afflicts) 
(GROUP_CONCAT(DISTINCT ?hasCauseLabel ; SEPARATOR = ",") AS  ?hasCauses) 
#(GROUP_CONCAT(DISTINCT(?altLabelLabel); separator = ", ") AS ?altLabel_list) 
{
  ?item wdt:P31 wd:Q12136;
        wdt:P699 ?doid;
        wdt:P279 ?subClass.
  OPTIONAL {?item wdt:P2293 ?geneticassociation.}
  OPTIONAL {?item wdt:P486 ?MeSH.}
  OPTIONAL {?item wdt:P1995 ?healthspecialty.}
  OPTIONAL {?item wdt:P780 ?symptom.}
  OPTIONAL {?item wdt:P828 ?hasCause.}
  OPTIONAL {?item wdt:P689 ?afflict.}
  #OPTIONAL{?item skos:altLabel ?altLabel . FILTER (lang(?altLabel) = "en") .}

  SERVICE wikibase:label { bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en". 
                         ?MeSH rdfs:label ?MeSHID.
                         ?doid rdfs:label ?DoidID.
                         ?subClass rdfs:label ?subClassLabel.
                         ?healthspecialty rdfs:label ?healthspecialtyLabel.
                         ?item rdfs:label ?itemName.
                         ?symptom rdfs:label ?symptomLabel.
                         ?geneticassociation rdfs:label ?geneticassociationLabel.
                         ?afflict rdfs:label ?afflictLabel.
                         ?hasCause rdfs:label ?hasCauseLabel.
                         #?altLabel rdfs:label ?altLabelLabel.
                         }
}GROUP BY ?item ?itemName ?MeSHID ?DoidID  

In [ ]:
# ATENÇÃO: Este comando é para introduzir no Wikidata Query service, encontrado em: https://query.wikidata.org/
# Esta query dá todas as doenças com um doid identifier, e nomes alternativos pelos quais podem ser conhecidas.

PREFIX dbo: <http://dbpedia.org/ontology/>
# disesases
SELECT 
?item 
?doid
?mesh

(GROUP_CONCAT(DISTINCT(?altLabelLabel); separator = ", ") AS ?altLabel_list) 
{
  ?item wdt:P31 wd:Q12136;
        wdt:P699 ?doid;
        wdt:P279 ?subClass.
  OPTIONAL{?item wdt:P486 ?mesh.}
  OPTIONAL{?item wdt:P4970 ?also.}
  OPTIONAL{?item skos:altLabel ?altLabel . FILTER (lang(?altLabel) = "en") }
  


  SERVICE wikibase:label { bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en".
                         ?altLabel rdfs:label ?altLabelLabel.}
         
}GROUP BY ?item ?doid ?mesh  

#### Com os queries em cima, criámos dois ficheiros csv, com informação acerca de doenças humanas. De seguida, processamos estes ficheiros como dataframes, para fundir e filtrar informação.
## Atenção, o ficheiro csv final já está criado, pelo que não é necessário fazer este passo de novo.

In [ ]:
import pandas as pd

#Import csv files as DataFrames and reset index to be able to fuse dataframes by index.
allDiseases = pd.read_csv('sparQL_query_all_diseases.csv')
itemIndexAllDiseases = allDiseases.set_index('item')
# itemIndexAllDiseases.head()

allDiseasesAltLabel = pd.read_csv('sparQL_query_altLabel.csv')
itemIndexAltLabels = allDiseasesAltLabel.set_index('item')
# itemIndexAltLabels.head()

# Merge the two dataframes by index and reset index.
mergedDf = itemIndexAllDiseases.merge(itemIndexAltLabels, left_index=True, right_index=True)
mergedAllDiseases = mergedDf.reset_index()
# mergedAllDiseases.head()

# Filter to remove unwanted columns and duplicates.
mergedAllDiseases = mergedAllDiseases.drop(columns=['doid', 'mesh'])
mergedAllDiseases = mergedAllDiseases.drop_duplicates()
# mergedAllDiseases.head()

# Create a cvs file with the filtered information:
mergedAllDiseases.to_csv(r'allDiseases.csv', index = False)

### Código para ler, filtrar e modelar dados acerca de doenças.

In [37]:
import csv

def load_csv(csv_file):
    '''This function reads a csv file with disease information compiled from Wikidata and it creates a list of lists with disease data.
        Requires: A csv file with a pre-determined structure.
        ENsures: Returns a list of list with all the disease data, ready to insert into a dimension table.
    '''
    
    with open(csv_file, newline='', encoding="utf8") as csv_file:
        csv_reader = csv.reader(csv_file, delimiter=',')
        data = list(csv_reader)
        
        diseases = []
        
        id = 0
        
        new_header = ['Disease_Key','Wikidata', 'Disease_Name', 'Doid_ID', 'MeSH_ID', 'Type_of_Disease', 'Health_Specialities', 'Symptoms', 'Genetic_Association', 'Affects', 'Caused', 'Alternative_Names']
        diseases.append(new_header)
        
        for item in data[1:]:
            id += 1
            wikidata = item[0].replace('http://www.wikidata.org/entity/','')
            name = item[1]
            doid = item[2].replace('DOID:','')
            mesh = item[3]
            subclass = item[4]
            healthspecialtie = item[5]
            symptoms = item[6]
            geneticassociations = '{' + item[7] + '}'
            affects = item[8]
            hascauses = item[9]
            alternativeName = item[10]
            
            diseases.append([id,wikidata,name,doid,mesh,subclass,healthspecialtie,symptoms,geneticassociations,affects,hascauses,alternativeName])
        
        return diseases
            
           
diseases = (load_csv("allDiseases.csv"))   

In [38]:
# Visualização da informação que será inserida da Disease_Dimension
diseases_DF = pd.DataFrame(diseases[1:],columns=diseases[0])
diseases_DF.head()

,Disease_Key,Wikidata,Disease_Name,Doid_ID,MeSH_ID,Type_of_Disease,Health_Specialities,Symptoms,Genetic_Association,Affects,Caused,Alternative_Names
0,1,Q1002195,autosomal recessive limb-girdle muscular dystr...,0110297,,autosomal recessive limb-girdle muscular dystr...,neurology,,{POMT1},,,"LGMD2K, limb-girdle muscular dystrophy-intelle..."
1,2,Q1004647,bullous pemphigoid,8506,D010391,"pemphigoid,autoimmune disease of skin and conn...",dermatology,,{},,,"Bullous pemphigoid, Bullous pemphigoid (disord..."
2,3,Q1016605,Burkitt lymphoma,8584,D002051,mature B-cell neoplasm,hematology,,{MYC},,Epstein Barr virus infection,"Burkitt lymphoma/leukaemia, Burkitt's Lymphoma..."
3,4,Q1017169,Buruli ulcer disease,0050456,D054312,"neglected tropical disease,mycobacterium infec...",infectious disease,,{},,infection,"Buruli ulcer, Bairnsdale ulcer, Daintree ulcer..."
4,5,Q1018534,progressive familial intrahepatic cholestasis,0070221,,"liver disease,intrahepatic cholestasis,familia...",,,"{ABCB11,ABCB4,ATP8B1}",,,PFIC; Byler disease


### Inserção e verificação dos dados na Disease_Dimension.

In [53]:
#SQL = """drop table disease_dimension"""
SQL= """create table disease_dimension(
disease_key int,
wikidata_id varchar(30),
disease_name varchar(160),
doid_id int, 
mesh_id varchar(20),
type_of_disease varchar(1760),
health_speciality  varchar(160),
symptoms varchar(1860),
genetic_associations varchar(60) ARRAY,
affects varchar(280),
caused_by varchar(480),
alternative_names varchar(20000),
primary key (disease_key)
)"""

excuteSingleSQLstatement(SQL, host="appserver-01.alunos.di.fc.ul.pt",database="tpd020", user="tpd020", password="tpd020_1000genomes")

In [39]:
conn = pg.connect(host="appserver-01.alunos.di.fc.ul.pt",database="tpd020", user="tpd020", password="tpd020_1000genomes")
curs = conn.cursor()
for dis in diseases[1:]:
    id, wikidata, name, doid, mesh, subclass, healthspecialtie, symptoms, geneticassociations, affects, hascauses, alternativeName = dis
    curs.execute("insert into disease_dimension values(%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)", (id, wikidata, name, doid, mesh, subclass, healthspecialtie, symptoms, geneticassociations, affects, hascauses, alternativeName))    
curs.close()
conn.commit()
conn.close()

In [40]:
SQL = "select * from disease_dimension"
df  = getSQLfromQuery(SQL, (), host="appserver-01.alunos.di.fc.ul.pt",database="tpd020", user="tpd020", password="tpd020_1000genomes")
df

,disease_key,wikidata_id,disease_name,doid_id,mesh_id,type_of_disease,health_speciality,symptoms,genetic_associations,affects,caused_by,alternative_names
0,1,Q1002195,autosomal recessive limb-girdle muscular dystr...,110297,,autosomal recessive limb-girdle muscular dystr...,neurology,,[POMT1],,,"LGMD2K, limb-girdle muscular dystrophy-intelle..."
1,2,Q1004647,bullous pemphigoid,8506,D010391,"pemphigoid,autoimmune disease of skin and conn...",dermatology,,[],,,"Bullous pemphigoid, Bullous pemphigoid (disord..."
2,3,Q1016605,Burkitt lymphoma,8584,D002051,mature B-cell neoplasm,hematology,,[MYC],,Epstein Barr virus infection,"Burkitt lymphoma/leukaemia, Burkitt's Lymphoma..."
3,4,Q1017169,Buruli ulcer disease,50456,D054312,"neglected tropical disease,mycobacterium infec...",infectious disease,,[],,infection,"Buruli ulcer, Bairnsdale ulcer, Daintree ulcer..."
4,5,Q1018534,progressive familial intrahepatic cholestasis,70221,,"liver disease,intrahepatic cholestasis,familia...",,,"[ABCB11, ABCB4, ATP8B1]",,,PFIC; Byler disease
...,...,...,...,...,...,...,...,...,...,...,...,...
9468,9469,Q992139,human granulocytic anaplasmosis,50025,,"tick-borne disease,anaplasmosis,ehrlichiosis,I...",infectious disease,,[],,infection,"HGE, human granulocytic ehrlichiosis"
9469,9470,Q994794,noma,9672,D009625,"aphthous stomatitis,bacterial infectious disea...",gastroenterology,,[],,,"cancrum oris, gangrenous stomatitis, necotizin..."
9470,9471,Q994859,homocystinuria,9263,D006712,"amino acid metabolic disorder,sulfuraminoacidemia",endocrinology,,[CBS],,,"CBS deficiency, cystathionine beta synthase de..."
9471,9472,Q994942,bruxism,2846,D002012,"sleep disorder,parafunctional habit",dentistry,dental attrition,[],,,"bruxism, Bruxism - teeth grinding, Grinding te..."


## SNP_Dimension and Study_Dimension

- As duas dimensões SNP e Study, são obtidas conjuntamente, pois a dimensão SNP depende dos dados que produzem a Study Dimension.
- A modelação destas dimensões foi muito complicada e o código ainda não se encontra optimizado.

In [41]:
conn = ''
def connectToServer():
    global conn
    conn = pg.connect(host="appserver-01.alunos.di.fc.ul.pt",database="tpd020", user="tpd020", password="tpd020_1000genomes")

- Vamos primeiro buscar a informação dos SNPs ao NCBI dbSNP.

In [110]:
import pandas as pd 
import re

i = 0
snpInfoList = []
snpInfo = []
with open('SNPS_Chr19.txt') as f:
    for l in f:
        #print(l)
        l = l.strip()
        if re.match("^\d+\.", l):
            if snpInfo != []:
                #print(snpInfoList)
                snpInfoList.append(snpInfo)
            snpInfo = []
            snp = l.split(' ')[1]
            snpInfo.append(snp)
        #elif l.startswith("Chromosome"):
        #    chromosome,snp_position = l.split(" ")[1].split(':')
        #    snpInfo.append(chromosome)
        #    snpInfo.append(snp_position)
        elif l.startswith("Gene"):
            gene = l.split(":")[1]
            snpInfo.append(gene)
        elif l.startswith("Functional Consequence"):
            functional_consequence = l.split(" ")[2]
            snpInfo.append(functional_consequence)
        elif l.startswith("Clinical significance"):
            clinical_significance = l.split(" ")[2]
            snpInfo.append(clinical_significance)
        i += 1
        #if i > 100:
        #   break
#print(snpLists)

snps_chr19 = pd.DataFrame(snpInfoList,columns=['snp_ref_code','gene','functional_consequence','clinical_significance'])
#snps_chr19 = pd.DataFrame(snpInfoList,columns=['snp_ref_code','chromosome','snp_position','gene','functional_consequence','clinical_significance'])
snps_chr19

,snp_ref_code,gene,functional_consequence,clinical_significance
0,rs688,LDLR,"intron_variant,synonymous_variant,coding_seque...","benign,benign-likely-benign"
1,rs2962,INSR,"missense_variant,synonymous_variant,coding_seq...",benign
2,rs2963,INSR,"synonymous_variant,coding_sequence_variant",benign
3,rs3008,JAK3,"3_prime_UTR_variant,genic_downstream_transcrip...",benign
4,rs3556,PEPD,3_prime_UTR_variant,benign
...,...,...,...,...
2450175,rs576173908,FBN3,intron_variant,None
2450176,rs576173926,TPM4,downstream_transcript_variant,None
2450177,rs576173983,SIPA1L3,intron_variant,None
2450178,rs576173984,ADGRE1,intron_variant,None


In [43]:
snps_chr19.head()

,snp_ref_code,gene,functional_consequence,clinical_significance
0,rs688,LDLR,"intron_variant,synonymous_variant,coding_seque...","benign,benign-likely-benign"
1,rs2962,INSR,"missense_variant,synonymous_variant,coding_seq...",benign
2,rs2963,INSR,"synonymous_variant,coding_sequence_variant",benign
3,rs3008,JAK3,"3_prime_UTR_variant,genic_downstream_transcrip...",benign
4,rs3556,PEPD,3_prime_UTR_variant,benign


- De seguida, vamos buscar a informação dos SNPs ao ficheiro dos 1000 genomes project.
- Estes dados foram previamente filtrados, para eliminar a informação genética de cada individuo e facilitar a introdução dos dados na tabela de dimensão SNP.

In [48]:
snp_filtered = pd.read_csv("SNP_filtered.csv") 

In [49]:
snp_filtered.columns

Index(['CHROM', 'POS', 'ID', 'REF', 'ALT_1', 'EAS_AF_1', 'EUR_AF_1',
       'AFR_AF_1', 'AMR_AF_1', 'SAS_AF_1'],
      dtype='object')

In [50]:
snp_filtered.columns = ['chromosome', 'snp_position', 'snp_ref_code', 'reference', 'alternative', 'easian_frequency', 'european_frequency',
       'african_frequency', 'american_frequency', 'sasian_frequency']

In [51]:
snp_filtered

,chromosome,snp_position,snp_ref_code,reference,alternative,easian_frequency,european_frequency,african_frequency,american_frequency,sasian_frequency
0,19,60842,rs541392352,A,G,0.0000,0.0000,0.0000,0.0014,0.0010
1,19,64705,rs559839262,C,T,0.0000,0.0030,0.0091,0.0000,0.0000
2,19,69984,rs372156287,G,A,0.0079,0.0159,0.0287,0.0231,0.0092
3,19,80844,rs546022921,T,C,0.0000,0.0020,0.0000,0.0000,0.0000
4,19,80849,rs564694216,G,C,0.0109,0.0000,0.0015,0.0000,0.0000
...,...,...,...,...,...,...,...,...,...,...
1751873,19,59118740,rs546075210,T,A,0.0000,0.0000,0.0008,0.0000,0.0000
1751874,19,59118764,rs556756243,G,A,0.0020,0.0000,0.0000,0.0000,0.0000
1751875,19,59118779,rs141816674,G,C,0.6498,0.3072,0.8835,0.4553,0.4315
1751876,19,59118783,rs150801216,C,G,0.6498,0.3072,0.8835,0.4553,0.4315


- Juntamos os dois dataframes num só.

In [52]:
df1 = snps_chr19.set_index('snp_ref_code').join(snp_filtered.set_index('snp_ref_code'),how="inner")
df1

#df1 = pd.merge(snps_chr19, snp_filtered, on='snp_ref_code', how='right')
#df1

,gene,functional_consequence,clinical_significance,chromosome,snp_position,reference,alternative,easian_frequency,european_frequency,african_frequency,american_frequency,sasian_frequency
snp_ref_code,,,,,,,,,,,,
rs10001,STXBP2,"coding_sequence_variant,genic_downstream_trans...",benign,19,7711221,T,C,0.7034,0.4036,0.5356,0.4669,0.3865
rs1000237,GATAD2A,"intron_variant,genic_upstream_transcript_variant",None,19,19518316,T,A,0.3185,0.3370,0.6619,0.4813,0.5348
rs1000244,COPE,upstream_transcript_variant,None,19,19030467,A,G,0.0000,0.0000,0.0174,0.0014,0.0000
rs1000245,COPE,"intron_variant,upstream_transcript_variant",None,19,19031087,C,G,0.0000,0.0000,0.0174,0.0014,0.0000
rs1000262,RPL18A,upstream_transcript_variant,None,19,17970627,C,G,0.2470,0.2286,0.0628,0.3040,0.2280
...,...,...,...,...,...,...,...,...,...,...,...,...
rs999814,IQCN,intron_variant,None,19,18368969,G,A,0.2371,0.2724,0.0189,0.3314,0.1646
rs999815,IQCN,intron_variant,None,19,18368988,G,A,0.2381,0.2704,0.0204,0.2017,0.1626
rs999850,DPY19L3,"intron_variant,genic_upstream_transcript_variant",None,19,32901045,T,A,0.6399,0.6402,0.4342,0.4265,0.5920


In [53]:
df1 = df1.reset_index()
df1

,snp_ref_code,gene,functional_consequence,clinical_significance,chromosome,snp_position,reference,alternative,easian_frequency,european_frequency,african_frequency,american_frequency,sasian_frequency
0,rs10001,STXBP2,"coding_sequence_variant,genic_downstream_trans...",benign,19,7711221,T,C,0.7034,0.4036,0.5356,0.4669,0.3865
1,rs1000237,GATAD2A,"intron_variant,genic_upstream_transcript_variant",None,19,19518316,T,A,0.3185,0.3370,0.6619,0.4813,0.5348
2,rs1000244,COPE,upstream_transcript_variant,None,19,19030467,A,G,0.0000,0.0000,0.0174,0.0014,0.0000
3,rs1000245,COPE,"intron_variant,upstream_transcript_variant",None,19,19031087,C,G,0.0000,0.0000,0.0174,0.0014,0.0000
4,rs1000262,RPL18A,upstream_transcript_variant,None,19,17970627,C,G,0.2470,0.2286,0.0628,0.3040,0.2280
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1706559,rs999814,IQCN,intron_variant,None,19,18368969,G,A,0.2371,0.2724,0.0189,0.3314,0.1646
1706560,rs999815,IQCN,intron_variant,None,19,18368988,G,A,0.2381,0.2704,0.0204,0.2017,0.1626
1706561,rs999850,DPY19L3,"intron_variant,genic_upstream_transcript_variant",None,19,32901045,T,A,0.6399,0.6402,0.4342,0.4265,0.5920
1706562,rs999854,CACNA1A,intron_variant,None,19,13616206,G,A,0.2569,0.0109,0.0076,0.2320,0.0706


- De seguida, vamos ler o dataset do GWAS catalogue e extrair a informação acerca dos SNPs identificados em cada estudo.

In [58]:
gwas_catalog = pd.read_csv("gwas_catalog_v1.0.2-associations_e98_r2020-03-08.tsv",sep='\t')
gwas_catalog.columns

#gwas_catalog.head()

Index(['DATE ADDED TO CATALOG', 'PUBMEDID', 'FIRST AUTHOR', 'DATE', 'JOURNAL',
       'LINK', 'STUDY', 'DISEASE/TRAIT', 'INITIAL SAMPLE SIZE',
       'REPLICATION SAMPLE SIZE', 'REGION', 'CHR_ID', 'CHR_POS',
       'REPORTED GENE(S)', 'MAPPED_GENE', 'UPSTREAM_GENE_ID',
       'DOWNSTREAM_GENE_ID', 'SNP_GENE_IDS', 'UPSTREAM_GENE_DISTANCE',
       'DOWNSTREAM_GENE_DISTANCE', 'STRONGEST SNP-RISK ALLELE', 'SNPS',
       'MERGED', 'SNP_ID_CURRENT', 'CONTEXT', 'INTERGENIC',
       'RISK ALLELE FREQUENCY', 'P-VALUE', 'PVALUE_MLOG', 'P-VALUE (TEXT)',
       'OR or BETA', '95% CI (TEXT)', 'PLATFORM [SNPS PASSING QC]', 'CNV',
       'MAPPED_TRAIT', 'MAPPED_TRAIT_URI', 'STUDY ACCESSION',
       'GENOTYPING TECHNOLOGY'],
      dtype='object')

In [59]:
gwas_catalog.shape

(179364, 38)

- O dataset é processado para remover as colunas não necessárias. 
- Para a simplificação do processo, removemos também casos em que os estudos declaravam mais que um SNP nos estudos de associação. Além disso, limitámos a nossa análise aos estudos de associação genética com SNPs localizados no cromossoma 19 (seguindo as decisões feitas na elaboração do projecto).
- Procedemos a mais filtragens dos dados, para garantir que a terminologia referente aos SNPs estava na forma rs*.

In [60]:
del gwas_catalog["DATE ADDED TO CATALOG"]
del gwas_catalog["FIRST AUTHOR"]
del gwas_catalog["JOURNAL"]
del gwas_catalog["LINK"]
del gwas_catalog["STUDY"]
del gwas_catalog["INITIAL SAMPLE SIZE"]
del gwas_catalog["REPLICATION SAMPLE SIZE"]
del gwas_catalog["CHR_ID"]
del gwas_catalog["MERGED"]
del gwas_catalog["SNP_ID_CURRENT"]
del gwas_catalog["PVALUE_MLOG"]
del gwas_catalog["P-VALUE (TEXT)"]
del gwas_catalog["OR or BETA"]
del gwas_catalog["95% CI (TEXT)"]
del gwas_catalog["PLATFORM [SNPS PASSING QC]"]
del gwas_catalog["CNV"]
del gwas_catalog["MAPPED_TRAIT"]
del gwas_catalog["MAPPED_TRAIT_URI"]
del gwas_catalog["STUDY ACCESSION"]
del gwas_catalog["GENOTYPING TECHNOLOGY"]

gwas_catalog.columns

Index(['PUBMEDID', 'DATE', 'DISEASE/TRAIT', 'REGION', 'CHR_POS',
       'REPORTED GENE(S)', 'MAPPED_GENE', 'UPSTREAM_GENE_ID',
       'DOWNSTREAM_GENE_ID', 'SNP_GENE_IDS', 'UPSTREAM_GENE_DISTANCE',
       'DOWNSTREAM_GENE_DISTANCE', 'STRONGEST SNP-RISK ALLELE', 'SNPS',
       'CONTEXT', 'INTERGENIC', 'RISK ALLELE FREQUENCY', 'P-VALUE'],
      dtype='object')

In [61]:
gwas_catalog.columns = ['pumbmed_id', 'study_date', 'disease', 'region', 'chr_pos', 'reported_genes', 'mapped_genes',
       'upstream_gene_id', 'downstream_gene_id', 'snp_gene_ids', 'upstream_gene_distance', 'dowstream_gene_distance',
        'strongest_snp_risk_allele','snps','context','intergenic','risk_allele_frequency','p_value']

In [62]:
gwas_catalog['multiple_snp_semicolon'] = gwas_catalog['snps'].str.contains(';')
gwas_catalog = gwas_catalog[gwas_catalog['multiple_snp_semicolon'] == False]
gwas_catalog['multiple_snp_colon'] = gwas_catalog['snps'].str.contains(',')
gwas_catalog = gwas_catalog[gwas_catalog['multiple_snp_colon'] == False]
gwas_catalog[gwas_catalog ['snps'].str.match('rs')]
gwas_catalog = gwas_catalog[gwas_catalog.region.notnull()]
gwas_catalog = gwas_catalog[gwas_catalog ['region'].str.match('19')]
gwas_catalog['p_value'] = gwas_catalog['p_value'].astype(float)


gwas_catalog

,pumbmed_id,study_date,disease,region,chr_pos,reported_genes,mapped_genes,upstream_gene_id,downstream_gene_id,snp_gene_ids,upstream_gene_distance,dowstream_gene_distance,strongest_snp_risk_allele,snps,context,intergenic,risk_allele_frequency,p_value,multiple_snp_semicolon,multiple_snp_colon
10,21102463,2010-11-21,Crohn's disease,19p13.3,1124836,"GPX4, SBNO2","SBNO2, SBNO2",NaN,NaN,"ENSG00000064932, ENSG00000278788",NaN,NaN,rs740495-G,rs740495,intron_variant,0.0,0.247,8.000000e-12,False,False
33,21102463,2010-11-21,Crohn's disease,19p13.2,10359299,"ICAM3, TYK2, ICAM1",TYK2,NaN,NaN,ENSG00000105397,NaN,NaN,rs12720356-G,rs12720356,missense_variant,0.0,0.084,1.000000e-12,False,False
40,21102463,2010-11-21,Crohn's disease,19q13.33,48711017,"FUT2, RASIP1",FUT2 - MAMSTR,ENSG00000176920,ENSG00000176909,NaN,5066.0,1708.0,rs281379-A,rs281379,intergenic_variant,1.0,0.487,7.000000e-12,False,False
110,21658281,2011-06-10,Sudden cardiac arrest,19q13.32,45804148,RSHL1,RSPH6A,NaN,NaN,ENSG00000104941,NaN,NaN,rs8111071-?,rs8111071,intron_variant,0.0,0.09,4.000000e-06,False,False
118,19118814,2009-01-03,Alzheimer's disease,19q13.31,44110055,ZNF224,AC021092.1,NaN,NaN,ENSG00000186019,NaN,NaN,rs2061333-?,rs2061333,intron_variant,0.0,NR,2.000000e-06,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
178774,30038396,2018-07-23,Highest math class taken (MTAG),19q13.11,32493978,Intergenic,DPY19L3 - PDCD5,ENSG00000178904,ENSG00000105185,NaN,8083.0,87212.0,rs7250367-A,rs7250367,intergenic_variant,1.0,0.8239,4.000000e-08,False,False
178775,30038396,2018-07-23,Highest math class taken (MTAG),19q13.12,35746344,Intergenic,"PSENEN, AD000671.1",NaN,NaN,"ENSG00000205155, ENSG00000188223",NaN,NaN,rs2293688-C,rs2293688,intron_variant,0.0,0.6589,5.000000e-10,False,False
178776,30038396,2018-07-23,Highest math class taken (MTAG),19q13.2,40589394,Intergenic,SHKBP1,NaN,NaN,ENSG00000160410,NaN,NaN,rs76000727-A,rs76000727,intron_variant,0.0,0.089,3.000000e-08,False,False
178777,30038396,2018-07-23,Highest math class taken (MTAG),19q13.42,53939797,Intergenic,CACNG7,NaN,NaN,ENSG00000105605,NaN,NaN,rs307913-A,rs307913,intron_variant,0.0,0.8516,6.000000e-12,False,False


In [63]:
gwas_catalog['snp_length_str'] = gwas_catalog['snps'].str.len()
gwas_catalog.sort_values('snp_length_str',ascending=False,inplace=True)
gwas_catalog

,pumbmed_id,study_date,disease,region,chr_pos,reported_genes,mapped_genes,upstream_gene_id,downstream_gene_id,snp_gene_ids,...,dowstream_gene_distance,strongest_snp_risk_allele,snps,context,intergenic,risk_allele_frequency,p_value,multiple_snp_semicolon,multiple_snp_colon,snp_length_str
64907,30698716,2019-01-29,HDL cholesterol levels in current drinkers,19p13.2,8364439,ANGPTL4,ANGPTL4,NaN,NaN,ENSG00000167772,...,NaN,rs116843064-A,rs116843064,missense_variant,0.0,NR,2.000000e-14,False,False,11
91016,29777097,2018-05-18,Family history of Alzheimer's disease,19q13.32,44842530,NR,BCAM - NECTIN2,ENSG00000187244,ENSG00000130202,NaN,...,3645.0,rs111371860-?,rs111371860,intergenic_variant,1.0,NR,2.000000e-18,False,False,11
125304,26053186,2015-06-08,3-hydroxypropylmercapturic acid levels in smokers,19q13.42,54650073,NR,"LILRB4, LILRB4, LILRB4, LILRB4",NaN,NaN,"ENSG00000186818, ENSG00000276042, ENSG00000278...",...,NaN,rs139806220-?,rs139806220,intron_variant,0.0,NR,5.000000e-08,False,False,11
90992,29875488,2018-06-06,Blood protein levels,19p13.2,7700507,NR,FCER2,NaN,NaN,ENSG00000104921,...,NaN,rs113893384-C,rs113893384,intron_variant,0.0,0.706,5.000000e-19,False,False,11
91009,29777097,2018-05-18,Family history of Alzheimer's disease,19q13.32,44723891,NR,AC243964.3,NaN,NaN,ENSG00000266903,...,NaN,rs150820726-?,rs150820726,intron_variant,0.0,NR,3.000000e-08,False,False,11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
89032,29507422,2018-03-05,Total cholesterol levels,19p13.2,11116926,NR,LDLR,NaN,NaN,ENSG00000130164,...,NaN,rs688-C,rs688,synonymous_variant,0.0,0.578,1.000000e-25,False,False,5
158350,29507422,2018-03-05,Low density lipoprotein cholesterol levels,19p13.2,11116926,NR,LDLR,NaN,NaN,ENSG00000130164,...,NaN,rs688-C,rs688,synonymous_variant,0.0,NR,2.000000e-32,False,False,5
158349,29507422,2018-03-05,Low density lipoprotein cholesterol levels,19p13.2,11116926,NR,LDLR,NaN,NaN,ENSG00000130164,...,NaN,rs688-C,rs688,synonymous_variant,0.0,0.578,5.000000e-30,False,False,5
89033,29507422,2018-03-05,Total cholesterol levels,19p13.2,11116926,NR,LDLR,NaN,NaN,ENSG00000130164,...,NaN,rs688-C,rs688,synonymous_variant,0.0,NR,4.000000e-28,False,False,5


In [64]:
del gwas_catalog["multiple_snp_semicolon"]
del gwas_catalog["multiple_snp_colon"]
del gwas_catalog["snp_length_str"]

- Fundimos a informação do dataframe dos SNPs (gerada anteriormente), com a informação acerca dos SNPs derivada do dataset GWAS.

In [67]:
df2 = df1.set_index('snp_ref_code').join(gwas_catalog.set_index('snps'))

df2 = df2.reset_index()
df2.head()

,index,gene,functional_consequence,clinical_significance,chromosome,snp_position,reference,alternative,easian_frequency,european_frequency,...,upstream_gene_id,downstream_gene_id,snp_gene_ids,upstream_gene_distance,dowstream_gene_distance,strongest_snp_risk_allele,context,intergenic,risk_allele_frequency,p_value
0,rs10001,STXBP2,"coding_sequence_variant,genic_downstream_trans...",benign,19,7711221,T,C,0.7034,0.4036,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,rs1000237,GATAD2A,"intron_variant,genic_upstream_transcript_variant",None,19,19518316,T,A,0.3185,0.3370,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,rs1000244,COPE,upstream_transcript_variant,None,19,19030467,A,G,0.0000,0.0000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,rs1000245,COPE,"intron_variant,upstream_transcript_variant",None,19,19031087,C,G,0.0000,0.0000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,rs1000262,RPL18A,upstream_transcript_variant,None,19,17970627,C,G,0.2470,0.2286,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [68]:
df2.columns

Index(['index', 'gene', 'functional_consequence', 'clinical_significance',
       'chromosome', 'snp_position', 'reference', 'alternative',
       'easian_frequency', 'european_frequency', 'african_frequency',
       'american_frequency', 'sasian_frequency', 'pumbmed_id', 'study_date',
       'disease', 'region', 'chr_pos', 'reported_genes', 'mapped_genes',
       'upstream_gene_id', 'downstream_gene_id', 'snp_gene_ids',
       'upstream_gene_distance', 'dowstream_gene_distance',
       'strongest_snp_risk_allele', 'context', 'intergenic',
       'risk_allele_frequency', 'p_value'],
      dtype='object')

In [69]:
del df2["pumbmed_id"]
del df2["study_date"]
del df2["strongest_snp_risk_allele"]
del df2["risk_allele_frequency"]
del df2["p_value"]

In [70]:
df2['chromosome'] = df2['chromosome'].astype(int)

df2

,index,gene,functional_consequence,clinical_significance,chromosome,snp_position,reference,alternative,easian_frequency,european_frequency,...,chr_pos,reported_genes,mapped_genes,upstream_gene_id,downstream_gene_id,snp_gene_ids,upstream_gene_distance,dowstream_gene_distance,context,intergenic
0,rs10001,STXBP2,"coding_sequence_variant,genic_downstream_trans...",benign,19,7711221,T,C,0.7034,0.4036,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,rs1000237,GATAD2A,"intron_variant,genic_upstream_transcript_variant",None,19,19518316,T,A,0.3185,0.3370,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,rs1000244,COPE,upstream_transcript_variant,None,19,19030467,A,G,0.0000,0.0000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,rs1000245,COPE,"intron_variant,upstream_transcript_variant",None,19,19031087,C,G,0.0000,0.0000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,rs1000262,RPL18A,upstream_transcript_variant,None,19,17970627,C,G,0.2470,0.2286,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1710170,rs999814,IQCN,intron_variant,None,19,18368969,G,A,0.2371,0.2724,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1710171,rs999815,IQCN,intron_variant,None,19,18368988,G,A,0.2381,0.2704,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1710172,rs999850,DPY19L3,"intron_variant,genic_upstream_transcript_variant",None,19,32901045,T,A,0.6399,0.6402,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1710173,rs999854,CACNA1A,intron_variant,None,19,13616206,G,A,0.2569,0.0109,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


- O processo de introdução no SQL dos cerca de 1.7 milhões de SNPs é muito moroso. Por essa razão, optamos por remover todas as instâncias de SNP que sabíamos não estar associados a doenças (por informação derivada da Study_dimension).
- Isto dá-nos 6557 SNPs no cromossoma 19, que têm uma associação genética identificada na Study Dimension.
- Idealmente, todos os SNPs deveriam ter sido introduzidos na SNP Dimension.

In [71]:
df2 = df2[df2.disease.notnull()]
del df2["disease"]

In [72]:
df2

,index,gene,functional_consequence,clinical_significance,chromosome,snp_position,reference,alternative,easian_frequency,european_frequency,...,chr_pos,reported_genes,mapped_genes,upstream_gene_id,downstream_gene_id,snp_gene_ids,upstream_gene_distance,dowstream_gene_distance,context,intergenic
53,rs1003531,DOT1L,intron_variant,None,19,2206575,G,T,0.1687,0.3241,...,2206576,NR,DOT1L,NaN,NaN,ENSG00000104885,NaN,NaN,intron_variant,0.0
57,rs10038,CRTC1,"3_prime_UTR_variant,genic_downstream_transcrip...",None,19,18892728,C,A,0.1042,0.2823,...,18781918,COMP,CRTC1,NaN,NaN,ENSG00000105662,NaN,NaN,3_prime_UTR_variant,0.0
70,rs1004246,None,None,None,19,16456260,C,T,0.6062,0.7296,...,1.63454e+07,NaN,KLF2 - AC020917.3,ENSG00000127528,ENSG00000277825,NaN,17575.0,7013.0,intergenic_variant,1.0
90,rs10048489,None,None,None,19,40343129,G,C,0.1012,0.0825,...,39852489,FCGBP,FBL - FCGBP,ENSG00000280548,ENSG00000281123,NaN,6110.0,10834.0,intergenic_variant,1.0
140,rs1005636,None,None,None,19,43806559,T,C,0.9990,0.8201,...,43302407,NR,CEACAMP11 - CEACAMP4,ENSG00000236123,ENSG00000230681,NaN,18089.0,764.0,intergenic_variant,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1709685,rs9797885,TMEM91,"intron_variant,genic_upstream_transcript_variant",None,19,41873001,A,G,0.4464,0.6750,...,41367096,NR,"AC011462.1, TMEM91",NaN,NaN,"ENSG00000255730, ENSG00000142046",NaN,NaN,intron_variant,0.0
1709721,rs9807841,QTRT1,intron_variant,None,19,10817656,T,C,0.7242,0.6024,...,10706980,QTRT1,QTRT1,NaN,NaN,ENSG00000213339,NaN,NaN,intron_variant,0.0
1709995,rs995834,None,None,None,19,28866596,G,A,0.5109,0.7137,...,28375689,LOC100420587,AC005580.1,NaN,NaN,ENSG00000267320,NaN,NaN,intron_variant,0.0
1710023,rs9967620,None,None,None,19,46415696,C,T,0.0873,0.1779,...,45912438,GIPR,MYPOP - NANOS2,ENSG00000176182,ENSG00000188425,NaN,9825.0,776.0,intergenic_variant,1.0


In [73]:
gwas_catalog['reported_genes'] = '{' + gwas_catalog['reported_genes'].astype(str) + '}'
gwas_catalog['mapped_genes'] = '{' + gwas_catalog['mapped_genes'].astype(str) + '}'
gwas_catalog['snp_gene_ids'] = '{' + gwas_catalog['snp_gene_ids'].astype(str) + '}'

In [74]:
df2 = df2.rename(columns={"index": "snp_ref_code"})

In [75]:
del df2['reported_genes']
del df2['mapped_genes']
del df2['snp_gene_ids']
del df2['intergenic']

In [76]:
print(df2.columns)
df2['context']

Index(['snp_ref_code', 'gene', 'functional_consequence',
       'clinical_significance', 'chromosome', 'snp_position', 'reference',
       'alternative', 'easian_frequency', 'european_frequency',
       'african_frequency', 'american_frequency', 'sasian_frequency', 'region',
       'chr_pos', 'upstream_gene_id', 'downstream_gene_id',
       'upstream_gene_distance', 'dowstream_gene_distance', 'context'],
      dtype='object')


53              intron_variant
57         3_prime_UTR_variant
70          intergenic_variant
90          intergenic_variant
140         intergenic_variant
                  ...         
1709685         intron_variant
1709721         intron_variant
1709995         intron_variant
1710023     intergenic_variant
1710121         intron_variant
Name: context, Length: 6557, dtype: object

- O código seguinte, insere os dados filtrados dos estudos GWAS, na Study Dimension.

In [77]:
connectToServer()
curs = conn.cursor()

cols = ",".join([str(i) for i in gwas_catalog.columns.tolist()])

lines = 0
for i,row in gwas_catalog.iterrows():
    try:
        sql = "INSERT INTO study_dim (" + cols + ") VALUES (" + "%s,"*(len(row)-1) + "%s)"
        #print(sql, tuple(row))
        curs.execute(sql, tuple(row))
        lines += 1
        if lines % 101:
            conn.commit()
        #if lines > 38:
        #    break
    except Exception as e:
        print(e,tuple(row))

curs.close()
conn.commit()
conn.close()

In [78]:
SQL = "select * from study_dim"
df  = getSQLfromQuery(SQL, (), host="appserver-01.alunos.di.fc.ul.pt",database="tpd020", user="tpd020", password="tpd020_1000genomes")
df

,study_id,pumbmed_id,study_date,disease,region,chr_pos,reported_genes,mapped_genes,upstream_gene_id,downstream_gene_id,snp_gene_ids,upstream_gene_distance,dowstream_gene_distance,strongest_snp_risk_allele,snps,snp_id_current,context,intergenic,risk_allele_frequency,p_value
0,1,30698716,2019-01-29,HDL cholesterol levels in current drinkers,19p13.2,8364439,[ANGPTL4],[ANGPTL4],NaN,NaN,[ENSG00000167772],NaN,NaN,rs116843064-A,rs116843064,None,missense_variant,0.0,NR,2.000000e-14
1,2,29777097,2018-05-18,Family history of Alzheimer's disease,19q13.32,44842530,[NR],[BCAM - NECTIN2],ENSG00000187244,ENSG00000130202,[nan],21109.0,3645.0,rs111371860-?,rs111371860,None,intergenic_variant,1.0,NR,2.000000e-18
2,3,26053186,2015-06-08,3-hydroxypropylmercapturic acid levels in smokers,19q13.42,54650073,[NR],"[LILRB4, LILRB4, LILRB4, LILRB4]",NaN,NaN,"[ENSG00000186818, ENSG00000276042, ENSG0000027...",NaN,NaN,rs139806220-?,rs139806220,None,intron_variant,0.0,NR,5.000000e-08
3,4,29875488,2018-06-06,Blood protein levels,19p13.2,7700507,[NR],[FCER2],NaN,NaN,[ENSG00000104921],NaN,NaN,rs113893384-C,rs113893384,None,intron_variant,0.0,0.706,5.000000e-19
4,5,29777097,2018-05-18,Family history of Alzheimer's disease,19q13.32,44723891,[NR],[AC243964.3],NaN,NaN,[ENSG00000266903],NaN,NaN,rs150820726-?,rs150820726,None,intron_variant,0.0,NR,3.000000e-08
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6812,6813,29507422,2018-03-05,Total cholesterol levels,19p13.2,11116926,[NR],[LDLR],NaN,NaN,[ENSG00000130164],NaN,NaN,rs688-C,rs688,None,synonymous_variant,0.0,0.578,1.000000e-25
6813,6814,29507422,2018-03-05,Low density lipoprotein cholesterol levels,19p13.2,11116926,[NR],[LDLR],NaN,NaN,[ENSG00000130164],NaN,NaN,rs688-C,rs688,None,synonymous_variant,0.0,NR,2.000000e-32
6814,6815,29507422,2018-03-05,Low density lipoprotein cholesterol levels,19p13.2,11116926,[NR],[LDLR],NaN,NaN,[ENSG00000130164],NaN,NaN,rs688-C,rs688,None,synonymous_variant,0.0,0.578,5.000000e-30
6815,6816,29507422,2018-03-05,Total cholesterol levels,19p13.2,11116926,[NR],[LDLR],NaN,NaN,[ENSG00000130164],NaN,NaN,rs688-C,rs688,None,synonymous_variant,0.0,NR,4.000000e-28


- O código seguinte, insere os dados acerca dos SNPs (filtrados, como mencionado previamente), na SNP dimension.

In [79]:
connectToServer()
curs = conn.cursor()

cols = ",".join([str(i) for i in df2.columns.tolist()])

lines = 0
for i,row in df2.iterrows():
    try:
        sql = "INSERT INTO snp_dim (" + cols + ") VALUES (" + "%s,"*(len(row)-1) + "%s)"
        #print(sql, tuple(row))
        curs.execute(sql, tuple(row))
        lines += 1
        if lines % 101:
            conn.commit()
        #if lines > 2:
        #    break
    except Exception as e:
        print(e,tuple(row))

curs.close()
conn.commit()
conn.close()


In [82]:
SQL = "select * from snp_dim"
df  = getSQLfromQuery(SQL, (), host="appserver-01.alunos.di.fc.ul.pt",database="tpd020", user="tpd020", password="tpd020_1000genomes")
df

,snp_id,snp_ref_code,gene,functional_consequence,clinical_significance,chromosome,snp_position,reference,alternative,info_string,...,sasian_frequency,variation_type,consequence,mapped_gene,upstream_gene_id,downstream_gene_id,upstream_gene_distance,dowstream_gene_distance,region,chr_pos
0,1,rs1003531,DOT1L,intron_variant,None,19.0,2206575,G ...,T ...,None,...,0.1626,None,None,None,NaN,NaN,NaN,NaN,19p13.3,2206576
1,2,rs10038,CRTC1,"3_prime_UTR_variant,genic_downstream_transcrip...",None,19.0,18892728,C ...,A ...,None,...,0.1442,None,None,None,NaN,NaN,NaN,NaN,19p13.11,18781918
2,3,rs1004246,None,None,None,19.0,16456260,C ...,T ...,None,...,0.8006,None,None,None,ENSG00000127528,ENSG00000277825,17575.0,7013.0,19p13.11,16345449.0
3,4,rs10048489,None,None,None,19.0,40343129,G ...,C ...,None,...,0.0358,None,None,None,ENSG00000280548,ENSG00000281123,6110.0,10834.0,19q13.2,39852489
4,5,rs1005636,None,None,None,19.0,43806559,T ...,C ...,None,...,0.9110,None,None,None,ENSG00000236123,ENSG00000230681,18089.0,764.0,19q13.31,43302407
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6552,6553,rs9797885,TMEM91,"intron_variant,genic_upstream_transcript_variant",None,19.0,41873001,A ...,G ...,None,...,0.6207,None,None,None,NaN,NaN,NaN,NaN,19q13.2,41367096
6553,6554,rs9807841,QTRT1,intron_variant,None,19.0,10817656,T ...,C ...,None,...,0.7239,None,None,None,NaN,NaN,NaN,NaN,19p13.2,10706980
6554,6555,rs995834,None,None,None,19.0,28866596,G ...,A ...,None,...,0.6350,None,None,None,NaN,NaN,NaN,NaN,19q12,28375689
6555,6556,rs9967620,None,None,None,19.0,46415696,C ...,T ...,None,...,0.1646,None,None,None,ENSG00000176182,ENSG00000188425,9825.0,776.0,19q13.32,45912438


### c. Describe the process for inserting facts data

O seguinte processo será usado para inserir dados nas tabelas de factos:

#### **SNP_Person_FACT** 
- Carregaríamos todos os dataframes das dimensões associadas com a fact table (person_dimension, population_dimension, gender_dimension, chromosome_dimension, allele_dimension). 
- De seguida, usando o dataset de variação genética do 1000 genomes project, iríamos começar a ler a primeira linha, referente ao primeiro SNP no cromossoma 19. A referência do cromossoma (que neste exercício académico é sempre o cromossoma 19) e do SNP, seria usada para determinar as chromosome_key no chromosome_dimension e a SNP_key na SNP dimension.
- De seguida, essa linha (que contém informação genética acerca de todos os indíviduos no estudo) seria analisada para identificar o primeiro indíviduo (person_code). Esse person code, seria usado para determinar o person_key da Person Dimension.
- A informação na Person_Dimension, permitiria chegar à population_key e gender_key (na person_dimension, temos informação acerca da população, país e género).
- Para finalizar, a informação genética do indíviduo, seria lida para determinar o genótipo de cada um dos dois alelos do indíviduo (a informação genética representa-se como 0|0, 0|1, 1|0 ou 1|1. 0 representa o alelo de referência, 1 o alelo alternativo).
#### Este processo permitiria introduzir a primeira linha de factos. Isto seria feito iterativamente, para inserir todos os factos na tabela.

#### **GWAS_Disease FACT** 
- Carregaríamos todos os dataframes das dimensões associadas com a facts table (snp_dimension, disease_dimension e study_dimension). 
- Usando o dataframe do study_dimension, definiriamos o study_key para introduzir na tabela de factos (refere um estudo realizado).
- A informação acerca da doença (no mesmo dataframe), seria usada para encontrar a doença na disease_dimension e, consequentemente, o disease_key. 
- Ainda a partir da dataframe da study_dimension, iríamos identificar o SNP_Reference_code, que iríamos usar para identificar o snp_key, na snp_dimension.
- Para finalizar, já com a informação atómica contida nas chaves study_key, disease_key, snp_key, iríamos adicionar o p-value presente na study_dimension, como uma medida não aditiva do facto.
#### Este processo permitiria introduzir a primeira linha de factos. Isto seria feito iterativamente, para inserir todos os factos na tabela.


## Código para inserção de factos na Disease_gwas_fact table.

- Como as tabelas de SQL das dimensões são pequenas, vamos carregá-las com a informação essencial para identificar os factos.
- De seguida, críamos queries sobre esses dataframes e produzimos uma função que produz uma nova dataframe que é usada para preencher a Disease GWAS fact table.

In [92]:
SQL = "select study_id,snps,disease,p_value from study_dim"
study_dim  = getSQLfromQuery(SQL, (), host="appserver-01.alunos.di.fc.ul.pt",database="tpd020", user="tpd020", password="tpd020_1000genomes")
SQL = "select snp_id,snp_ref_code from snp_dim"
snp_dim  = getSQLfromQuery(SQL, (), host="appserver-01.alunos.di.fc.ul.pt",database="tpd020", user="tpd020", password="tpd020_1000genomes")
SQL = "select disease_key,disease_name,alternative_names from disease_dimension"
diseases_dimension  = getSQLfromQuery(SQL, (), host="appserver-01.alunos.di.fc.ul.pt",database="tpd020", user="tpd020", password="tpd020_1000genomes")

In [93]:
study_dim.columns

Index(['study_id', 'snps', 'disease', 'p_value'], dtype='object')

In [94]:
try:
    snp_id = snp_dim.loc[snp_dim['snp_ref_code']=='rs1003531']['snp_id'].values[0]
except:
    snp_id = None
#df.loc[df['column_name'] == some_value]
snp_id

1

In [95]:
my_query = 'byssinosis'

disease_id =  diseases_dimension[diseases_dimension ['disease_name'].str.match(my_query,case=False)]['disease_key'].values[0]

disease_id
                      
#disease_name = test3.loc[test3['disease_name']==my_query]
#if disease_name.empty:
#    disease_name =  test3[test3 ['alternative_names'].str.match('.*'+my_query+'.*',case=False)]
#disease_name['disease_key'].values[0]
                      
#my_disease_key = disease_name['disease_key'].values[0]



#print('###',my_disease_key)

6

In [97]:
snp_facts = []
for i,row in study_dim.iterrows():
    study_id,snps,disease,p_value = row["study_id"],row["snps"],row["disease"],row["p_value"]
    #print(study_id,snps,disease,p_value)
    try:
        snp_id = snp_dim.loc[snp_dim['snp_ref_code']==snps]['snp_id'].values[0]
    except:
        snp_id = ''
    try:
        disease_id =  diseases_dimension[diseases_dimension ['disease_name'].str.match(disease,case=False)]['disease_key'].values[0]
    except:
        disease_id = ''
    #if disease_name.empty:
    #    disease_name =  test3[test3 ['alternative_names'].str.match('.*'+disease+'.*',case=False)]
    if snp_id != '' and disease_id != '':
        #print(study_id,snp_id,disease_id)
        snp_facts.append([study_id,snp_id,disease_id,p_value])
    
snp_facts_df = pd.DataFrame(snp_facts,columns=['study_id','snp_id','disease_id','p_value'])

snp_facts_df
    
    

,study_id,snp_id,disease_id,p_value
0,18,1066,302,5.000000e-07
1,31,671,302,7.000000e-09
2,40,1095,918,7.000000e-09
3,102,1053,7014,3.000000e-23
4,133,652,7160,2.000000e-08
...,...,...,...,...
527,6744,6400,302,9.000000e-13
528,6781,5742,9273,3.000000e-39
529,6784,5742,9273,3.000000e-39
530,6785,5742,9273,2.000000e-35


In [98]:
snp_facts_df.columns

Index(['study_id', 'snp_id', 'disease_id', 'p_value'], dtype='object')

- A função seguinte, insere a dataframe na tabela de factos.

In [107]:
connectToServer()
curs = conn.cursor()

cols = ",".join([str(i) for i in snp_facts_df.columns.tolist()])

lines = 0
for i,row in snp_facts_df.iterrows():
    try:
        sql = "INSERT INTO disease_gwas_fact (" + cols + ") VALUES (" + "%s,"*(len(row)-1) + "%s)"
        #print(sql, tuple(row))
        curs.execute(sql, tuple(row))
        lines += 1
        if lines % 101:
            conn.commit()
        #if lines > 2:
        #    break
    except Exception as e:
        print(e,tuple(row))

curs.close()
conn.commit()
conn.close()

In [108]:
SQL = "select * from disease_gwas_fact"
df  = getSQLfromQuery(SQL, (), host="appserver-01.alunos.di.fc.ul.pt",database="tpd020", user="tpd020", password="tpd020_1000genomes")
df

,disease_id,snp_id,study_id,p_value
0,302,1066,18,5.000000e-07
1,302,671,31,7.000000e-09
2,918,1095,40,7.000000e-09
3,7014,1053,102,3.000000e-23
4,7160,652,133,2.000000e-08
...,...,...,...,...
527,302,6400,6744,9.000000e-13
528,9273,5742,6781,3.000000e-39
529,9273,5742,6784,3.000000e-39
530,9273,5742,6785,2.000000e-35


## Código para inserção de factos na snp_person_fact table.

In [ ]:
conn = ''
def connectToServer():
    global conn
    conn = pg.connect(host="appserver-01.alunos.di.fc.ul.pt",database="tpd020", user="tpd020", password="tpd020_1000genomes")

- Primeiro vamos por as tabelas de sql mais pequenas em dataframes.

In [113]:
SQL = "select allele_key,allele_type from allele_dimension"
allele_dim = getSQLfromQuery(SQL, (), host="appserver-01.alunos.di.fc.ul.pt",database="tpd020", user="tpd020", password="tpd020_1000genomes")
allele_dim

,allele_key,allele_type
0,0,Reference
1,1,Alternative


- Esta dimensão não tem chave substituta, vamos usar a allele_key directamente

In [114]:
SQL = "select snp_id,snp_ref_code from snp_dim"
snp_dim = getSQLfromQuery(SQL, (), host="appserver-01.alunos.di.fc.ul.pt",database="tpd020", user="tpd020", password="tpd020_1000genomes")
snp_dim.head()

,snp_id,snp_ref_code
0,1,rs1003531
1,2,rs10038
2,3,rs1004246
3,4,rs10048489
4,5,rs1005636


In [115]:
try:
    snp_id = snp_dim.loc[snp_dim['snp_ref_code']=='rs10048489']['snp_id'].values[0]
except:
    snp_id = None
snp_id

4

- Para os snps optámos por fazer uma chamada SQL em vez de converter em dataframe. Isto porque, quando completa, esta tabela teria ~2milhões de entradas.

In [116]:
sql = "select snp_id from snp_dim where snp_ref_code = %s"
snp_dim = getSQLfromQuery(sql, ('rs10048489',), host="appserver-01.alunos.di.fc.ul.pt",database="tpd020", user="tpd020", password="tpd020_1000genomes")
try: 
    snp_id = snp_dim['snp_id'].values[0]
except:
    snp_id = None
snp_id

4

In [117]:
SQL = "select chromosome_key,chromosome from chromosome_dimension"
chromosome_dimension = getSQLfromQuery(SQL, (), host="appserver-01.alunos.di.fc.ul.pt",database="tpd020", user="tpd020", password="tpd020_1000genomes")
chromosome_dimension.head()

,chromosome_key,chromosome
0,1,1
1,2,2
2,3,3
3,4,4
4,5,5


In [118]:
try:
    chromosome_id = chromosome_dimension.loc[chromosome_dimension['chromosome']=='19']['chromosome_key'].values[0]
except:
    chromosome_id = None
chromosome_id

19

In [119]:
SQL = "select gender_key,gender,country_code from gender_dimension"
gender_dim = getSQLfromQuery(SQL, (), host="appserver-01.alunos.di.fc.ul.pt",database="tpd020", user="tpd020", password="tpd020_1000genomes")
gender_dim.head()

,gender_key,gender,country_code
0,1,male,BGD
1,2,female,BGD
2,7,male,COL
3,8,female,COL
4,9,male,ESP


In [120]:
gender = 'female'
country_code = 'COL'
try:
    gender_row = gender_dim.loc[(gender_dim['gender'] == gender) & (gender_dim['country_code'] == country_code)]
    gender_id = gender_row['gender_key'].values[0]
except:
    gender_id = None
    
gender_id
    

8

In [121]:
SQL = "select person_key,country_code,person_code,population_code,country,gender from person_dimension"
person_dimension = getSQLfromQuery(SQL, (), host="appserver-01.alunos.di.fc.ul.pt",database="tpd020", user="tpd020", password="tpd020_1000genomes")
person_dimension.head()

,person_key,country_code,person_code,population_code,country,gender
0,1,GBR,HG00096,GBR,United Kingdom,male
1,2,GBR,HG00097,GBR,United Kingdom,female
2,3,GBR,HG00099,GBR,United Kingdom,female
3,4,GBR,HG00100,GBR,United Kingdom,female
4,5,GBR,HG00101,GBR,United Kingdom,male


In [122]:
gender = 'female'
country_code = 'COL'
try:
    gender_row = gender_dim.loc[(gender_dim['gender'] == gender) & (gender_dim['country_code'] == country_code)]
    gender_id = gender_row['gender_key'].values[0]
except:
    gender_id = None
    
gender_id

8

In [123]:
person_code = 'HG00100'
try:
    person_row = person_dimension.loc[person_dimension['person_code']==person_code]
    #print(person_row)
    person_id = person_row['person_key'].values[0]
    population_code = person_row['population_code'].values[0]
    country_code = person_row['country_code'].values[0]
    gender = person_row['gender'].values[0]
except:
    person_id,population_code,country_code,gender = None,None,None
    
person_id,population_code,country_code,gender

(4, 'GBR', 'GBR', 'female')

In [124]:
SQL = "select population_key,population_code from population_dimension"
population_dim = getSQLfromQuery(SQL, (), host="appserver-01.alunos.di.fc.ul.pt",database="tpd020", user="tpd020", password="tpd020_1000genomes")
population_dim.head()

,population_key,population_code
0,1,CDX
1,2,CHB
2,3,JPT
3,4,KHV
4,5,CHS


In [125]:
population_code = 'JPT'
try:
    population_row = population_dim.loc[population_dim['population_code'] == population_code]
    population_key = population_row['population_key'].values[0]
except:
    population_key = None
    
population_key

3

- Sabendo já como fazer queries sobre todos os dataframes vamos criar um programa que traduz as product_keys em ids das tabelas dimensão, para inserer na facts table.

- Aqui limitámos o número de linhas que vamos processar a título de exemplo.

In [137]:
import pandas as pd
i = 0
fact_snp = []
with open("ALL.chr19.phase3_shapeit2_mvncall_integrated_v5a.20130502.genotypes.vcf") as f:
    for l in f:
        l = l.strip()
        if not l.startswith("##"):
            if i == 0:
                i += 1
                continue
            l = l.split("\t")
            curr_snp = l[2]
            curr_chrom = l[0]
            l = l[9:]
            n_entries = len(l)
            #print(l)
            if i == 252:
                my_columns = l
                #print(len(my_columns))
                #print(my_columns)
            if i > 252:
                for f in range(n_entries):
                    person_code = my_columns[f]
                    allele1,allele2 = l[f].split('|')
                    #print(curr_chrom,curr_snp,person_code,allele1,allele2)
                    try:
                        person_row = person_dimension.loc[person_dimension['person_code']==person_code]
                        person_id = person_row['person_key'].values[0]
                        population_code = person_row['population_code'].values[0]
                        country_code = person_row['country_code'].values[0]
                        gender = person_row['gender'].values[0]
                    except:
                        person_id,population_code,country_code,gender = None,None,None,None
                    try:
                        person_row = person_dimension.loc[person_dimension['person_code']==person_code]
                        person_id = person_row['person_key'].values[0]
                        population_code = person_row['population_code'].values[0]
                        gender = person_row['gender'].values[0]
                    except:
                        person_id,population_code,gender = None,None,None
                    try:
                        chromosome_id = chromosome_dimension.loc[chromosome_dimension['chromosome']==curr_chrom]['chromosome_key'].values[0]
                    except:
                        chromosome_id = None
                    try: 
                        sql = "select snp_id from snp_dim where snp_ref_code = %s"
                        snp_dim = getSQLfromQuery(sql, (curr_snp,), host="localhost",database="test1", user="eli", password="pwd")
                        snp_id = snp_dim['snp_id'].values[0]
                    except:
                        snp_id = None
                    try:
                        population_row = population_dim.loc[population_dim['population_code'] == population_code]
                        population_key = population_row['population_key'].values[0]
                    except:
                        population_key = None
                    try:
                        gender_row = gender_dim.loc[(gender_dim['gender'] == gender) & (gender_dim['country_code'] == country_code)]
                        gender_id = gender_row['gender_key'].values[0]
                    except:
                        gender_id = None
                    #print(curr_chrom,curr_snp,person_code,population_code,gender,allele1,allele2)
                    insert_me = [chromosome_id,snp_id,person_id,population_key,gender_id,allele1,allele2]
                    #print(insert_me)
                    fact_snp.append(insert_me)
                #print(l)
                #snps_alleles.append(l)
        i += 1
        if i > 255: #limitámos o número de linhas que vamos processar aqui
            break

snps_alleles_df = pd.DataFrame(fact_snp,columns=['chromossome_id','snp_id','person_id','population_id','gender_id','allele_1','allele_2'])

In [138]:
snps_alleles_df

,chromossome_id,snp_id,person_id,population_id,gender_id,allele_1,allele_2
0,19,None,1,18,13,0,0
1,19,None,2,18,14,0,0
2,19,None,3,18,14,0,0
3,19,None,4,18,14,0,0
4,19,None,5,18,13,0,0
...,...,...,...,...,...,...,...
7507,19,None,2500,7,34,0,0
7508,19,None,2501,7,34,0,0
7509,19,None,2502,7,34,0,0
7510,19,None,2503,7,34,0,0


- Finalmente inserimos o dataframe da tabela de factos.

In [139]:
connectToServer()
curs = conn.cursor()

cols = ",".join([str(i) for i in snps_alleles_df.columns.tolist()])

lines = 0
for i,row in snps_alleles_df.iterrows():
    try:
        sql = "INSERT INTO snp_person_fact (" + cols + ") VALUES (" + "%s,"*(len(row)-1) + "%s)"
        #print(sql, tuple(row))
        curs.execute(sql, tuple(row))
        lines += 1
        if lines % 101:
            conn.commit()
        #if lines > 2:
        #    break
    except Exception as e:
        print(e,tuple(row))

curs.close()
conn.commit()
conn.close()

In [140]:
SQL = "select * from snp_person_fact"
df  = getSQLfromQuery(SQL, (), host="appserver-01.alunos.di.fc.ul.pt",database="tpd020", user="tpd020", password="tpd020_1000genomes")
df

,gender_id,population_id,snp_id,person_id,chromossome_id,allele_1,allele_2
0,13,18,None,1,19,0,0
1,14,18,None,2,19,0,0
2,14,18,None,3,19,0,0
3,14,18,None,4,19,0,0
4,13,18,None,5,19,0,0
...,...,...,...,...,...,...,...
7507,34,7,None,2500,19,0,0
7508,34,7,None,2501,19,0,0
7509,34,7,None,2502,19,0,0
7510,34,7,None,2503,19,0,0


## 3. Do a critical assessment of the work
### a. Describe potential issues with the ETL procedure used

O processo de ETL foi um desafio árduo, visto que não foi possível encontrar um único repositório com todos os datasets/dados necessários. Projectos genómicos desta dimensão e âmbito, necessitam de aglomerar dados de várias fontes diferentes. Descarregar, organizar, filtrar, compatibilizar estes dados é um desafio enorme. Neste sentido, recorremos a dados de fontes como, NCBI (informação acerca de SNPs e cromossomas), 1000 genomes project, The World Data Bank, GWAS Catalogue e Wikidata. Tendo em conta a dimensão dos dados a agregar e a necessidade de os normalizar e compatibilizar, é fácil entender a complexidade deste exercício.

Os problemas que encontramos na obtenção de datasets adequados, podem ser ilustrados com o desafio em obter dados agregados acerca de doenças humanas. Na realidade, na nossa busca, o único repositório que encontramos que agrega informação acerca de doenças humanas é o site https://www.malacards.org/. Este site aglomera dados de 74 (!!) fontes distintas para obter informação rica acerca de doenças. Infelizmente, não há maneira de aceder aos datasets deste site. Chegamos inclusivamente a contactar o Help Desk, mais foi-nos dito que para obter os dados precisávamos de um acordo institucional. Esta pequena estória, exemplifica bem a importância de usar várias fontes para enriquecer os dados. Por isso mesmo, a maioria das nossas tabelas de dimensões resultam da fusão de dados de datasets diferentes (por exemplo, os dados para a tabela population Dimension, vieram de dados do dataset 1000 genomes e de datasets do World Data Bank). 

Voltando ao desafio com a dimensão Disesases (um dos grandes desafios, que nos levou, sem exagero, meses!), não se encontrou nenhum dataset que contivesse os dados necessários. Tentamos recorrer a ontologias como Doid e Human Diseases Ontologies, mas mesmo assim os dados eram "pobre". Após muita procura, acabamos por utilizar o Wikidata, uma solução "knowledge base", onde se encontraram todos os dados necessário para a dimensão. Para extrair e construir o dataset "caseiro" recorreu-se à ferramenta Wikidata Query Service na qual se utiliza a linguagem SPARQL para o desenvolvimento das queries, que nos permitiram obter os dados para carregar na nossa dimensão de doenças.

**A complexidade e dificuldade que foi o processo de "data acquisition", transpirou para o processo de ETL. De facto, vários problemas emergiram durante este processo de "escultura" e compatibilização de dados de diferentes fontes.**
Heis alguns dos problemas com os quais nos deparamos (alguns dos quais não foram completamente resolvidos):
- Redundância entre os dados de diferentes fontes, que tentamos normalizar o melhor que conseguimos.
- Problemas de ter campos em datasets com entradas múltiplas (a solução actual, foi transformar estes campos em arrays).
- Problemas de modelação das Study and Disease Dimension, nomeadamente no que diz respeito aos nomes das doenças. Muitas doenças têm nomes alternativos, pelo que optamos por incluir na Disease Dimension, um campo com "Alternative_Names". Infelizmente, este campo não foi ainda modelado da melhor forma e é uma solução a explorar no futuro.
- Problemas com indíces demográficos em falta para as dimensões Gender e Population. Preencher estes dados necessitaria de mais tempo e mais fontes de dados demográficos. De qualquer modo, este é um aspecto a considerar em versões futuras do DW, que serão guiadas pelos vários campos de "negócio", por forma a enriquecer de maneira substancial os dados demográficos.
- Na modelação dos dados demográficos, tivemos que decidir como modelar estes dados no tempo. Dado a natureza do DW actual, decidimos extrair os dados demográficos relativos ao ano em que o 1000 genomes project foi publicado, 2015. No futuro, poderemos melhorar esta solução, incluindo mais anos, médias das últimas décadas, adicionar indíces dos últimos anos disponíveis, etc.). Estes dados demográficos, se crescerem significativamente, podem inclusive ser modelados de maneira diferente, em outtriggers, em dimensões slow changing (SCD), etc...

**Dadas as dimensões como GWAS e SNP Dimension e a dificuldade em modular estas dimensões tomamos várias decisões estratégicas que se justificam pelo teor académico deste exercício:**
- Focamos a nossa análise genética só no cromossoma 19 (e que já assim são cerca de 1.8 milhões de linhas de SNPs).
- Existem vários tipos de variações genéticas (SNPs, CNVs, INDELs, etc.). Para simplificar a nossa análises usamos só dados de SNPs com um único alelo alternativo.
- No study Dimension usamos apenas os estudos de associação que identificam um único SNP no cromossoma 19 associado a uma determinada doença.
- Dada a dimensão da SNP dimension, optámos por modelar os nossos dados para inserir apenas a informação relativa a SNPs que foram reportados no GWAS catalogue, como tendo uma associação genética a uma doença (e que de facto constituem os dados que vão ser introduzidos na tabela de factos). 
- Na Study Dimension, estão incluídos estudos acerca de associação genética com doenças, mas também com outros traços e características humanas. Isto cria uma dificuldade acrescida, na selecção de dados de associação relacionados apenas com doenças.
- Devido à complexidade na modelação destas duas dimensões, resta ainda muito trabalho para fazer na optimização dos dados respectivos.

Devido a uma enorme esforço de coordenação, conseguiu-se solucionar a maioria dos problemas verificados durante o processo de ETL. Porém, a fiabilidade dos dados poderá não ser ainda a melhor (dados desatualizados e/ou com falhas). Próximas versões deste DW deverão concentrar-se em limpar e harmonizar os dados.

De salientar que houve uma decisão de modelação de última hora: por forma a lidar com os campos Allele_1 e Allele_2 na SNP_Person_Fact, criou-se uma Allele_Dimension, cujo único propósito é identificar se o alelo é o a referência, ou o alternativo. Achamos que esta "Role-Playing Dimension" irá ajudar na inteligibilidade da leitura do esquema, pelos end-users.

Por fim, acreditamos que os erros a evitar em modelação dimensional foram evitados.

#### Nota: 
- Um erro de última hora foi detectado quando tentámos inserir os factos na Disease GWAS fact. Detectámos uma incompatibilidade na nomenclatura usada para descrever as doenças nos dados obtidos do GWAS dataset e da Wikidata. Para resolver este problema, fomos forçados a recorrer à comparação por expressões regulares. Isto não é ideal mas permite-nos, para já, evitar falsos positivos. Os dados serão, no futuro, tratados para resolver este tipo de problemas.
- Nesta versão do Notebook, e após uma noite inteira a tentar resolver problemas de incompatibilidade com os dados, conseguimos criar e injectar a tabela de factos SNP Person Fact.
- O código para criação da Study Dimension e SNP Dimension está ainda muito complexo e necessita ser simplificado e melhorado. 

### b. Compare your schema to the one previously defined in phase I

O formato essencial do nosso esquema de dados, não mudou dramaticamente em comparação com aquele proposto na fase 1 do projecto. As mudanças operadas, resultaram da consolidação dos dados, do refinamento da informação guardada em cada dimensão, da conformação de dimensões, e da tentativa de melhor o DW. 

Heis alguns dos exemplos de inovação operados nesta fase do projecto:
- Vários dados demográficos foram adicionados às Gender e Population Dimensions. Este exercício académico, pretende ser apenas exemplificativo. Muitos mais dados e, uma muito maior riqueza do esquema, poderia ser facilmente atingida modelando uma maior quantidade de dados no nosso DW.
- Em quase todas as populações, aumentamos o número de colunas, por forma a facilitar a leitura e inteligibilidade do dados. Preocupamo-nos também em adicionar atributos de uma forma conformada, por forma a permitir a ligação entre dimensões no futuro (por exemplo, em exercícios de "drill-across" através de queries em diferentes tabelas de factos).
- Adicionamos vários parámetros métricos à Chromosome Dimension, e parámetros pré-calculados, como densidade de genes por megabase. 
- Adicionaram-se vários atributos à dimensão SNP, incluindo contexto genómico, consequências funcionais, genes associados.
- A Disease Dimension foi completamente renovada, com novos atributos derivados dos dados que obtemos.
- A Study Dimension agrega mais informação relativamente aos genes associados com os SNPs identificados no estudo e valores probabilisticos da associação.
- A Disesase GWAS Fact passou a ter uma medida não aditiva, o p-value derivado do estudo de associação genética. Este valor permite medir, para o evento atómico, a probabilidade de um  SNP estar associado com uma doença.
- A SNP Person Fact, passou a ser uma factless table. Esta decisão derivou da necessidade de modelar a informação genética relativamente aos alelos de cada indíviduo (para cada SNP). A informação que tínhamos planeado colocar em cada linha de factos era, para cada um dos dois alelos, o valor 0 representando um alelo de referência, ou o valor 1 representando um alelo alternativo. Como estas medidas seriam pouco informativas e difíceis de entender, críamos uma Allele_Dimension, onde a informação acerca do tipo de alelo (Reference ou Alternative) pode ser facilmente obtida.

### c. Discuss the issues for updating the data warehouse with novel data

O nosso Data Warehouse é relativamente limitado em termos de updates no tempo. Além do mais, não existem hierarquias nas dimensões, pelo que a modelação no tempo acaba por ser facilitada. Em termos de crescimento, este acontecerá apenas quando novos estudos GWAS forem publicados, ou novos dados genéticos estiverem disponíveis. Neste sentido, e no ponto de vista do exercício académico, estas alterações seriam periódicas e consistiriam de adicionar novas linhas às tabelas de dimensões respectivas (com consequente aumento dos factos, para cada estudo e para cada indíviduos cujo genoma fosse sequenciado). Por isso, a grande preocupação será a de garantir que novos dados são adicionados na ordem correcta (e que dados antigos não são reintroduzidos).

No que diz respeito a Slow Changing Dimensions (SCDs), as únicas SCD que podemos considerar são aquelas que agregam informação demográfica. Nos actuais esquemas de negócio, não existe interesse em guardar historial dos dados, pelo que estas dimensões comportar-se-iam com SCD do tipo 1, simplesmente substituíndo os indíces demográficos por valores actualizados. Se existisse a necessidade de se referir a dados demográficos históricos, poder-se-ia considerar o uso de SCDs de tipo IV, onde se usaria uma "mini-dimension" com o histórico dos dados (de 5 em 5 anos) e que não obrigaria a grandes alterações ao esquema actual.